## Importing Libraries

In [1]:
import argparse
import datetime as dte
import os
import json

import data_formatters.base
import expt_settings.configs
import libs.hyperparam_opt
import libs.tft_model
import libs.utils as utils
import numpy as np
import pandas as pd
import tensorflow.compat.v1 as tf

import warnings 

warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

ExperimentConfig = expt_settings.configs.ExperimentConfig
HyperparamOptManager = libs.hyperparam_opt.HyperparamOptManager
ModelClass = libs.tft_model.TemporalFusionTransformer
tf.experimental.output_all_intermediates(True)

### Experiment Metrics Tracking

In [2]:
with open('electricity_dataset_experiments.json', 'r') as f:
    loss_experiment_tracker = json.load(f)

dataset_name = "electricity"
dataset_folder_path = "electricity_dataset"

### Creating Model

In [3]:
loss_experiment_tracker[dataset_folder_path] = {}

name = dataset_name
output_folder = dataset_folder_path

use_tensorflow_with_gpu = True
print("Using output folder {}".format(output_folder))

config = ExperimentConfig(name, output_folder)
formatter = config.make_data_formatter()

expt_name=name
use_gpu=use_tensorflow_with_gpu
model_folder=os.path.join(config.model_folder, "fixed")
data_csv_path=config.data_csv_path
data_formatter=formatter
use_testing_mode=True

num_repeats = 1

if not isinstance(data_formatter, data_formatters.base.GenericDataFormatter):
    raise ValueError(
        "Data formatters should inherit from" +
        "AbstractDataFormatter! Type={}".format(type(data_formatter)))

# Tensorflow setup
default_keras_session = tf.keras.backend.get_session()

if use_gpu:
    tf_config = utils.get_default_tensorflow_config(tf_device="gpu", gpu_id=0)

else:
    tf_config = utils.get_default_tensorflow_config(tf_device="cpu")

print("*** Training from defined parameters for {} ***".format(expt_name))

print("Loading & splitting data...")
raw_data = pd.read_csv(data_csv_path, index_col=0)
train, valid, test = data_formatter.split_data(raw_data)
train_samples, valid_samples = data_formatter.get_num_samples_for_calibration()

# Sets up default params
fixed_params = data_formatter.get_experiment_params()
params = data_formatter.get_default_model_params()
params["model_folder"] = model_folder

# Parameter overrides for testing only! Small sizes used to speed up script.
if use_testing_mode:
    fixed_params["num_epochs"] = 1
    params["hidden_layer_size"] = 5
    train_samples, valid_samples = 100, 10

# Sets up hyperparam manager
print("*** Loading hyperparm manager ***")
opt_manager = HyperparamOptManager({k: [params[k]] for k in params},
                                   fixed_params, model_folder)

# Training -- one iteration only
print("*** Running calibration ***")
print("Params Selected:")

for k in params:
    print("{}: {}".format(k, params[k]))

best_loss = np.Inf

for _ in range(num_repeats):

    tf.reset_default_graph()
    with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

        tf.keras.backend.set_session(sess)

        params = opt_manager.get_next_parameters()
        model = ModelClass(params, use_cudnn=use_gpu)

        if not model.training_data_cached():
            model.cache_batched_data(train, "train", num_samples=train_samples)
            model.cache_batched_data(valid, "valid", num_samples=valid_samples)

        sess.run(tf.global_variables_initializer())
        model.fit()

        val_loss = model.evaluate()

        if val_loss < best_loss:
            opt_manager.update_score(params, val_loss, model)
            best_loss = val_loss

        tf.keras.backend.set_session(default_keras_session)

Using output folder electricity_dataset

Metal device set to: Apple M1 Pro
Selecting GPU ID=0
*** Training from defined parameters for electricity ***
Loading & splitting data...
Formatting train-valid-test splits.
Setting scalers with training data...
*** Loading hyperparm manager ***
*** Running calibration ***
Params Selected:
dropout_rate: 0.1
hidden_layer_size: 5
learning_rate: 0.001
minibatch_size: 64
max_gradient_norm: 0.01
num_heads: 4
stack_size: 1
model_folder: electricity_dataset/saved_models/electricity/fixed
Resetting temp folder...
*** TemporalFusionTransformer params ***
# dropout_rate = 0.1
# hidden_layer_size = 5
# learning_rate = 0.001
# max_gradient_norm = 0.01
# minibatch_size = 64
# model_folder = electricity_dataset/saved_models/electricity/fixed
# num_heads = 4
# stack_size = 1
# total_time_steps = 192
# num_encoder_steps = 168
# num_epochs = 1
# early_stopping_patience = 5
# multiprocessing_workers = 5
# column_definition = [('id', <DataTypes.REAL_VALUED: 0>, <I

                                                                  'multiply[0][0]']               
                                                                                                  
 multiply_1 (Multiply)          (None, 1, 5)         0           ['dense_11[0][0]',               
                                                                  'dense_12[0][0]']               
                                                                                                  
 layer_normalization (LayerNorm  (None, 1)           2           ['add[0][0]']                    
 alization)                                                                                       
                                                                                                  
 add_1 (Add)                    (None, 1, 5)         0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/strided_slice_11[0][0]',  
          

 tf_op_layer_TemporalFusionTran  [(None, 192, 5, 1)]  0          ['time_distributed[0][0]']       
 sformer/stack_1 (TensorFlowOpL                                                                   
 ayer)                                                                                            
                                                                                                  
 multiply_3 (Multiply)          (None, 5)            0           ['dense_15[0][0]',               
                                                                  'dense_16[0][0]']               
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 3)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_8 (Tenso                                  former/stack_2[0][0]']           
 rFlowOpLayer)                                                                                    
          

 ibuted)                                                         former/strided_slice_12[0][0]']  
                                                                                                  
 time_distributed_14 (TimeDistr  (None, 168, 5)      30          ['tf_op_layer_TemporalFusionTrans
 ibuted)                                                         former/strided_slice_13[0][0]']  
                                                                                                  
 time_distributed_18 (TimeDistr  (None, 168, 5)      30          ['tf_op_layer_TemporalFusionTrans
 ibuted)                                                         former/strided_slice_14[0][0]']  
                                                                                                  
 time_distributed_22 (TimeDistr  (None, 168, 5)      30          ['tf_op_layer_TemporalFusionTrans
 ibuted)                                                         former/strided_slice_15[0][0]']  
          

 dropout_9 (Dropout)            (None, 168, 5)       0           ['time_distributed_19[0][0]']    
                                                                                                  
 dropout_10 (Dropout)           (None, 168, 5)       0           ['time_distributed_23[0][0]']    
                                                                                                  
 dense_21 (Dense)               (None, 5)            30          ['tf_op_layer_TemporalFusionTrans
                                                                 former/Sum[0][0]']               
                                                                                                  
 dense_25 (Dense)               (None, 5)            30          ['tf_op_layer_TemporalFusionTrans
                                                                 former/Sum[0][0]']               
                                                                                                  
 dropout_1

...

 ibuted)                                                                                          
                                                                                                  
 time_distributed_50 (TimeDistr  (None, 192, 5)      30          ['dropout_16[0][0]']             
 ibuted)                                                                                          
                                                                                                  
 multiply_19 (Multiply)         (None, 192, 5)       0           ['time_distributed_49[0][0]',    
                                                                  'time_distributed_50[0][0]']    
                                                                                                  
 add_16 (Add)                   (None, 192, 5)       0           ['layer_normalization_15[0][0]', 
                                                                  'multiply_19[0][0]']            
          

                                                                 former/Cumsum[0][0]']            
                                                                                                  
 lambda_6 (Lambda)              (None, 192, 192)     0           ['dense_81[0][0]',               
                                                                  'dense_82[0][0]']               
                                                                                                  
 lambda_7 (Lambda)              (None, None, None)   0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Cumsum[0][0]']            
                                                                                                  
 lambda_9 (Lambda)              (None, 192, 192)     0           ['dense_83[0][0]',               
                                                                  'dense_84[0][0]']               
          

 multiply_20 (Multiply)         (None, 192, 5)       0           ['time_distributed_51[0][0]',    
                                                                  'time_distributed_52[0][0]']    
                                                                                                  
 add_21 (Add)                   (None, 192, 5)       0           ['multiply_20[0][0]',            
                                                                  'layer_normalization_16[0][0]'] 
                                                                                                  
 layer_normalization_17 (LayerN  (None, 192, 5)      10          ['add_21[0][0]']                 
 ormalization)                                                                                    
                                                                                                  
 time_distributed_53 (TimeDistr  (None, 192, 5)      30          ['layer_normalization_17[0][0]'] 
 ibuted)  

Extracting 100 samples...
Cached data "train" updated
Getting valid sampling locations.
Getting locations for MT_001
Getting locations for MT_002
Getting locations for MT_003
Getting locations for MT_004
Getting locations for MT_005
Getting locations for MT_006
Getting locations for MT_007
Getting locations for MT_008
Getting locations for MT_009
Getting locations for MT_010
Getting locations for MT_011
Getting locations for MT_012
Getting locations for MT_013
Getting locations for MT_014
Getting locations for MT_015
Getting locations for MT_016
Getting locations for MT_017
Getting locations for MT_018
Getting locations for MT_019
Getting locations for MT_020
Getting locations for MT_021
Getting locations for MT_022
Getting locations for MT_023
Getting locations for MT_024
Getting locations for MT_025
Getting locations for MT_026
Getting locations for MT_027
Getting locations for MT_028
Getting locations for MT_029
Getting locations for MT_030
Getting locations for MT_031
Getting locat

*** Fitting TemporalFusionTransformer ***
Getting batched_data
Using cached training data
Using cached validation data
Using keras standard fit
Train on 100 samples, validate on 10 samples
100/100 [==============================] - 13s 128ms/sample - loss: 2.5948 - val_loss: 2.5859
Cannot load from electricity_dataset/saved_models/electricity/fixed/tmp, skipping ...
Using cached validation data
Optimal model found, updating
INFO:tensorflow:electricity_dataset/saved_models/electricity/fixed/TemporalFusionTransformer.ckpt.index
INFO:tensorflow:0
INFO:tensorflow:electricity_dataset/saved_models/electricity/fixed/TemporalFusionTransformer.ckpt.meta
INFO:tensorflow:4300
INFO:tensorflow:electricity_dataset/saved_models/electricity/fixed/TemporalFusionTransformer.ckpt.data-00000-of-00001
INFO:tensorflow:4300
Model saved to: electricity_dataset/saved_models/electricity/fixed/TemporalFusionTransformer.ckpt


## Calculating Mean Absolute Error ( MAE )

In [4]:
from sklearn.metrics import mean_absolute_error

print("*** Running tests ***")
tf.reset_default_graph()

with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    best_params = opt_manager.get_best_params()
    model = ModelClass(best_params, use_cudnn=use_gpu)

    model.load(opt_manager.hyperparam_folder)

    print("Computing best validation loss")
    val_loss = model.evaluate(valid)

    print("Computing test loss")
    output_map = model.predict(test, return_targets=True)
    targets = data_formatter.format_predictions(output_map["targets"])

    p10_forecast = data_formatter.format_predictions(output_map["p10"])
    p50_forecast = data_formatter.format_predictions(output_map["p50"])
    p90_forecast = data_formatter.format_predictions(output_map["p90"])

    def extract_numerical_data(data):
        """Strips out forecast time and identifier columns."""
        return data[[
            col for col in data.columns
            if col not in {"forecast_time", "identifier"}
        ]]

    p10_loss = mean_absolute_error(extract_numerical_data(targets), extract_numerical_data(p10_forecast))
    p50_loss = mean_absolute_error(extract_numerical_data(targets), extract_numerical_data(p50_forecast))
    p90_loss = mean_absolute_error(extract_numerical_data(targets), extract_numerical_data(p90_forecast))

    tf.keras.backend.set_session(default_keras_session)

print("Training completed @ {}".format(dte.datetime.now()))
print("Best validation loss = {}".format(val_loss))
print("Params:")

for k in best_params:
    print(k, " = ", best_params[k])

    
print("*"*100)
print("*"*100)
print("Mean Absolute Error Loss for Test Data: P10={},  P50={}, P90={}".format(p10_loss.mean(), p50_loss.mean(), p90_loss.mean()))
print("*"*100)
print("*"*100)

loss_experiment_tracker[dataset_folder_path].update({
    "Mean Absolute Error p10 Loss": str(p10_loss.mean()),
    "Mean Absolute Error p50 Loss": str(p50_loss.mean()),
    "Mean Absolute Error p90 Loss": str(p90_loss.mean()),
})

*** Running tests ***
Resetting temp folder...
*** TemporalFusionTransformer params ***
# dropout_rate = 0.1
# hidden_layer_size = 5
# learning_rate = 0.001
# max_gradient_norm = 0.01
# minibatch_size = 64
# model_folder = electricity_dataset/saved_models/electricity/fixed
# num_heads = 4
# stack_size = 1
# total_time_steps = 192
# num_encoder_steps = 168
# num_epochs = 1
# early_stopping_patience = 5
# multiprocessing_workers = 5
# column_definition = [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('power_usage', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('hour', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('categorical_id', <DataTypes.CATEGORICAL: 1>, <InputTypes.STATIC_INPUT: 3>)]
# input_size = 5
# output_size = 1
# category_c

                                                                  'multiply_1[0][0]']             
                                                                                                  
 activation_1 (Activation)      (None, 1)            0           ['layer_normalization[0][0]']    
                                                                                                  
 layer_normalization_1 (LayerNo  (None, 1, 5)        10          ['add_1[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 1, 1)]      0           ['activation_1[0][0]']           
 sformer/ExpandDims (TensorFlow                                                                   
 OpLayer)                                                                                         
          

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 1)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_9 (Tenso                                  former/stack_1[0][0]']           
 rFlowOpLayer)                                                                                    
                                                                                                  
 add_2 (Add)                    (None, 5)            0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Sum[0][0]',               
                                                                  'multiply_3[0][0]']             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 4)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/c

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['time_distributed_27[0][0]',    
 sformer/add_11 (TensorFlowOpLa                                   'time_distributed_28[0][0]']    
 yer)                                                                                             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_16 (Tens                                  former/strided_slice_10[0][0]']  
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/s

 dropout_11 (Dropout)           (None, 24, 5)        0           ['time_distributed_29[0][0]']    
                                                                                                  
 time_distributed_33 (TimeDistr  (None, 24, 5)       30          ['activation_15[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_37 (TimeDistr  (None, 24, 5)       30          ['activation_16[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_41 (TimeDistr  (None, 24, 5)       30          ['activation_17[0][0]']          
 ibuted)                                                                                          
          

...

                                                                 former/Sum_2[0][0]',             
                                                                  'cu_dnnlstm[0][1]',             
                                                                  'cu_dnnlstm[0][2]']             
                                                                                                  
 dense_18 (Dense)               (None, 5)            30          ['activation_4[0][0]']           
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 192, 5)]    0           ['cu_dnnlstm[0][0]',             
 sformer/concat_2 (TensorFlowOp                                   'cu_dnnlstm_1[0][0]']           
 Layer)                                                                                           
                                                                                                  
 dropout_3

                                                                                                  
 layer_normalization_16 (LayerN  (None, 192, 5)      10          ['add_16[0][0]']                 
 ormalization)                                                                                    
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(3,)]              0           ['layer_normalization_16[0][0]'] 
 sformer/Shape (TensorFlowOpLay                                                                   
 er)                                                                                              
                                                                                                  
 tf_op_layer_TemporalFusionTran  [()]                0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_19 (Tens                                  former/Shape[0][0]']             
 orFlowOpL

                                                                                                  
 lambda_10 (Lambda)             (None, None, None)   0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Cumsum[0][0]']            
                                                                                                  
 add_17 (Add)                   (None, 192, 192)     0           ['lambda[0][0]',                 
                                                                  'lambda_1[0][0]']               
                                                                                                  
 add_18 (Add)                   (None, 192, 192)     0           ['lambda_3[0][0]',               
                                                                  'lambda_4[0][0]']               
                                                                                                  
 add_19 (A

 ibuted)                                                                                          
                                                                                                  
 activation_20 (Activation)     (None, 192, 5)       0           ['time_distributed_53[0][0]']    
                                                                                                  
 time_distributed_54 (TimeDistr  (None, 192, 5)      30          ['activation_20[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 dropout_24 (Dropout)           (None, 192, 5)       0           ['time_distributed_54[0][0]']    
                                                                                                  
 time_distributed_55 (TimeDistr  (None, 192, 5)      30          ['dropout_24[0][0]']             
 ibuted)  

INFO:tensorflow:Restoring parameters from electricity_dataset/saved_models/electricity/fixed/TemporalFusionTransformer.ckpt
Done.
Computing best validation loss
Computing test loss
Training completed @ 2022-09-24 23:45:09.348670
Best validation loss = 2.3729475421444284
Params:
dropout_rate  =  0.1
hidden_layer_size  =  5
learning_rate  =  0.001
max_gradient_norm  =  0.01
minibatch_size  =  64
model_folder  =  electricity_dataset/saved_models/electricity/fixed
num_heads  =  4
stack_size  =  1
total_time_steps  =  192
num_encoder_steps  =  168
num_epochs  =  1
early_stopping_patience  =  5
multiprocessing_workers  =  5
column_definition  =  [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('power_usage', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('hour', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_

## Calculating Mean Bias Error (MBE)

In [5]:
print("*** Running tests ***")
tf.reset_default_graph()

with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    best_params = opt_manager.get_best_params()
    model = ModelClass(best_params, use_cudnn=use_gpu)

    model.load(opt_manager.hyperparam_folder)

    print("Computing best validation loss")
    val_loss = model.evaluate(valid)

    print("Computing test loss")
    output_map = model.predict(test, return_targets=True)
    targets = data_formatter.format_predictions(output_map["targets"])

    p10_forecast = data_formatter.format_predictions(output_map["p10"])
    p50_forecast = data_formatter.format_predictions(output_map["p50"])
    p90_forecast = data_formatter.format_predictions(output_map["p90"])

    def extract_numerical_data(data):
        """Strips out forecast time and identifier columns."""
        return data[[
            col for col in data.columns
            if col not in {"forecast_time", "identifier"}
        ]]

    p10_loss = np.mean(extract_numerical_data(p10_forecast)-extract_numerical_data(targets))
    p50_loss = np.mean(extract_numerical_data(p50_forecast)-extract_numerical_data(targets))
    p90_loss = np.mean(extract_numerical_data(p90_forecast)-extract_numerical_data(targets))

    tf.keras.backend.set_session(default_keras_session)

print("Training completed @ {}".format(dte.datetime.now()))
print("Best validation loss = {}".format(val_loss))
print("Params:")

for k in best_params:
    print(k, " = ", best_params[k])

print("*"*100)
print("*"*100)
print("Mean Bias Error Loss for Test Data: P10={},  P50={}, P90={}".format(p10_loss.mean(), p50_loss.mean(), p90_loss.mean()))
print("*"*100)
print("*"*100)

loss_experiment_tracker[dataset_folder_path].update({
    "Mean Bias Error p10 Loss": str(p10_loss.mean()),
    "Mean Bias Error p50 Loss": str(p50_loss.mean()),
    "Mean Bias Error p90 Loss": str(p90_loss.mean()),
})

*** Running tests ***
Resetting temp folder...
*** TemporalFusionTransformer params ***
# dropout_rate = 0.1
# hidden_layer_size = 5
# learning_rate = 0.001
# max_gradient_norm = 0.01
# minibatch_size = 64
# model_folder = electricity_dataset/saved_models/electricity/fixed
# num_heads = 4
# stack_size = 1
# total_time_steps = 192
# num_encoder_steps = 168
# num_epochs = 1
# early_stopping_patience = 5
# multiprocessing_workers = 5
# column_definition = [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('power_usage', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('hour', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('categorical_id', <DataTypes.CATEGORICAL: 1>, <InputTypes.STATIC_INPUT: 3>)]
# input_size = 5
# output_size = 1
# category_c

                                                                  'multiply_1[0][0]']             
                                                                                                  
 activation_1 (Activation)      (None, 1)            0           ['layer_normalization[0][0]']    
                                                                                                  
 layer_normalization_1 (LayerNo  (None, 1, 5)        10          ['add_1[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 1, 1)]      0           ['activation_1[0][0]']           
 sformer/ExpandDims (TensorFlow                                                                   
 OpLayer)                                                                                         
          

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 1)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_9 (Tenso                                  former/stack_1[0][0]']           
 rFlowOpLayer)                                                                                    
                                                                                                  
 add_2 (Add)                    (None, 5)            0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Sum[0][0]',               
                                                                  'multiply_3[0][0]']             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 4)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/c

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['time_distributed_27[0][0]',    
 sformer/add_11 (TensorFlowOpLa                                   'time_distributed_28[0][0]']    
 yer)                                                                                             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_16 (Tens                                  former/strided_slice_10[0][0]']  
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/s

 dropout_11 (Dropout)           (None, 24, 5)        0           ['time_distributed_29[0][0]']    
                                                                                                  
 time_distributed_33 (TimeDistr  (None, 24, 5)       30          ['activation_15[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_37 (TimeDistr  (None, 24, 5)       30          ['activation_16[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_41 (TimeDistr  (None, 24, 5)       30          ['activation_17[0][0]']          
 ibuted)                                                                                          
          

...

                                                                 former/Sum_2[0][0]',             
                                                                  'cu_dnnlstm[0][1]',             
                                                                  'cu_dnnlstm[0][2]']             
                                                                                                  
 dense_18 (Dense)               (None, 5)            30          ['activation_4[0][0]']           
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 192, 5)]    0           ['cu_dnnlstm[0][0]',             
 sformer/concat_2 (TensorFlowOp                                   'cu_dnnlstm_1[0][0]']           
 Layer)                                                                                           
                                                                                                  
 dropout_3

                                                                                                  
 layer_normalization_16 (LayerN  (None, 192, 5)      10          ['add_16[0][0]']                 
 ormalization)                                                                                    
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(3,)]              0           ['layer_normalization_16[0][0]'] 
 sformer/Shape (TensorFlowOpLay                                                                   
 er)                                                                                              
                                                                                                  
 tf_op_layer_TemporalFusionTran  [()]                0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_19 (Tens                                  former/Shape[0][0]']             
 orFlowOpL

                                                                                                  
 lambda_10 (Lambda)             (None, None, None)   0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Cumsum[0][0]']            
                                                                                                  
 add_17 (Add)                   (None, 192, 192)     0           ['lambda[0][0]',                 
                                                                  'lambda_1[0][0]']               
                                                                                                  
 add_18 (Add)                   (None, 192, 192)     0           ['lambda_3[0][0]',               
                                                                  'lambda_4[0][0]']               
                                                                                                  
 add_19 (A

 ibuted)                                                                                          
                                                                                                  
 activation_20 (Activation)     (None, 192, 5)       0           ['time_distributed_53[0][0]']    
                                                                                                  
 time_distributed_54 (TimeDistr  (None, 192, 5)      30          ['activation_20[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 dropout_24 (Dropout)           (None, 192, 5)       0           ['time_distributed_54[0][0]']    
                                                                                                  
 time_distributed_55 (TimeDistr  (None, 192, 5)      30          ['dropout_24[0][0]']             
 ibuted)  

INFO:tensorflow:Restoring parameters from electricity_dataset/saved_models/electricity/fixed/TemporalFusionTransformer.ckpt
Done.
Computing best validation loss
Computing test loss
Training completed @ 2022-09-24 23:53:32.214554
Best validation loss = 2.3729475421444284
Params:
dropout_rate  =  0.1
hidden_layer_size  =  5
learning_rate  =  0.001
max_gradient_norm  =  0.01
minibatch_size  =  64
model_folder  =  electricity_dataset/saved_models/electricity/fixed
num_heads  =  4
stack_size  =  1
total_time_steps  =  192
num_encoder_steps  =  168
num_epochs  =  1
early_stopping_patience  =  5
multiprocessing_workers  =  5
column_definition  =  [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('power_usage', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('hour', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_

## Calculating Relative Absolute Error (RAE)

In [6]:
def rae(actual, predicted):
    numerator = np.sum(np.abs(predicted - actual))
    denominator = np.sum(np.abs(np.mean(actual) - actual))
    return numerator / denominator

print("*** Running tests ***")
tf.reset_default_graph()

with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    best_params = opt_manager.get_best_params()
    model = ModelClass(best_params, use_cudnn=use_gpu)

    model.load(opt_manager.hyperparam_folder)

    print("Computing best validation loss")
    val_loss = model.evaluate(valid)

    print("Computing test loss")
    output_map = model.predict(test, return_targets=True)
    targets = data_formatter.format_predictions(output_map["targets"])

    p10_forecast = data_formatter.format_predictions(output_map["p10"])
    p50_forecast = data_formatter.format_predictions(output_map["p50"])
    p90_forecast = data_formatter.format_predictions(output_map["p90"])

    def extract_numerical_data(data):
        """Strips out forecast time and identifier columns."""
        return data[[
            col for col in data.columns
            if col not in {"forecast_time", "identifier"}
        ]]

    p10_loss = rae(extract_numerical_data(targets), extract_numerical_data(p10_forecast))
    p50_loss = rae(extract_numerical_data(targets), extract_numerical_data(p50_forecast))
    p90_loss = rae(extract_numerical_data(targets), extract_numerical_data(p90_forecast))

    tf.keras.backend.set_session(default_keras_session)

print("Training completed @ {}".format(dte.datetime.now()))
print("Best validation loss = {}".format(val_loss))
print("Params:")

for k in best_params:
    print(k, " = ", best_params[k])

print("*"*100)
print("*"*100)
print("Relative Absolute Error Loss for Test Data: P10={},  P50={}, P90={}".format(p10_loss.mean(), p50_loss.mean(), p90_loss.mean()))
print("*"*100)
print("*"*100)

loss_experiment_tracker[dataset_folder_path].update({
    "Relative Absolute Error p10 Loss": str(p10_loss.mean()),
    "Relative Absolute Error p50 Loss": str(p50_loss.mean()),
    "Relative Absolute Error p90 Loss": str(p90_loss.mean()),
})

*** Running tests ***
Resetting temp folder...
*** TemporalFusionTransformer params ***
# dropout_rate = 0.1
# hidden_layer_size = 5
# learning_rate = 0.001
# max_gradient_norm = 0.01
# minibatch_size = 64
# model_folder = electricity_dataset/saved_models/electricity/fixed
# num_heads = 4
# stack_size = 1
# total_time_steps = 192
# num_encoder_steps = 168
# num_epochs = 1
# early_stopping_patience = 5
# multiprocessing_workers = 5
# column_definition = [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('power_usage', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('hour', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('categorical_id', <DataTypes.CATEGORICAL: 1>, <InputTypes.STATIC_INPUT: 3>)]
# input_size = 5
# output_size = 1
# category_c

                                                                  'multiply_1[0][0]']             
                                                                                                  
 activation_1 (Activation)      (None, 1)            0           ['layer_normalization[0][0]']    
                                                                                                  
 layer_normalization_1 (LayerNo  (None, 1, 5)        10          ['add_1[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 1, 1)]      0           ['activation_1[0][0]']           
 sformer/ExpandDims (TensorFlow                                                                   
 OpLayer)                                                                                         
          

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 1)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_9 (Tenso                                  former/stack_1[0][0]']           
 rFlowOpLayer)                                                                                    
                                                                                                  
 add_2 (Add)                    (None, 5)            0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Sum[0][0]',               
                                                                  'multiply_3[0][0]']             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 4)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/c

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['time_distributed_27[0][0]',    
 sformer/add_11 (TensorFlowOpLa                                   'time_distributed_28[0][0]']    
 yer)                                                                                             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_16 (Tens                                  former/strided_slice_10[0][0]']  
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/s

 dropout_11 (Dropout)           (None, 24, 5)        0           ['time_distributed_29[0][0]']    
                                                                                                  
 time_distributed_33 (TimeDistr  (None, 24, 5)       30          ['activation_15[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_37 (TimeDistr  (None, 24, 5)       30          ['activation_16[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_41 (TimeDistr  (None, 24, 5)       30          ['activation_17[0][0]']          
 ibuted)                                                                                          
          

...

                                                                 former/Sum_2[0][0]',             
                                                                  'cu_dnnlstm[0][1]',             
                                                                  'cu_dnnlstm[0][2]']             
                                                                                                  
 dense_18 (Dense)               (None, 5)            30          ['activation_4[0][0]']           
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 192, 5)]    0           ['cu_dnnlstm[0][0]',             
 sformer/concat_2 (TensorFlowOp                                   'cu_dnnlstm_1[0][0]']           
 Layer)                                                                                           
                                                                                                  
 dropout_3

                                                                                                  
 layer_normalization_16 (LayerN  (None, 192, 5)      10          ['add_16[0][0]']                 
 ormalization)                                                                                    
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(3,)]              0           ['layer_normalization_16[0][0]'] 
 sformer/Shape (TensorFlowOpLay                                                                   
 er)                                                                                              
                                                                                                  
 tf_op_layer_TemporalFusionTran  [()]                0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_19 (Tens                                  former/Shape[0][0]']             
 orFlowOpL

                                                                                                  
 lambda_10 (Lambda)             (None, None, None)   0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Cumsum[0][0]']            
                                                                                                  
 add_17 (Add)                   (None, 192, 192)     0           ['lambda[0][0]',                 
                                                                  'lambda_1[0][0]']               
                                                                                                  
 add_18 (Add)                   (None, 192, 192)     0           ['lambda_3[0][0]',               
                                                                  'lambda_4[0][0]']               
                                                                                                  
 add_19 (A

 ibuted)                                                                                          
                                                                                                  
 activation_20 (Activation)     (None, 192, 5)       0           ['time_distributed_53[0][0]']    
                                                                                                  
 time_distributed_54 (TimeDistr  (None, 192, 5)      30          ['activation_20[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 dropout_24 (Dropout)           (None, 192, 5)       0           ['time_distributed_54[0][0]']    
                                                                                                  
 time_distributed_55 (TimeDistr  (None, 192, 5)      30          ['dropout_24[0][0]']             
 ibuted)  

INFO:tensorflow:Restoring parameters from electricity_dataset/saved_models/electricity/fixed/TemporalFusionTransformer.ckpt
Done.
Computing best validation loss
Computing test loss
Training completed @ 2022-09-25 00:01:30.607034
Best validation loss = 2.3729475421444284
Params:
dropout_rate  =  0.1
hidden_layer_size  =  5
learning_rate  =  0.001
max_gradient_norm  =  0.01
minibatch_size  =  64
model_folder  =  electricity_dataset/saved_models/electricity/fixed
num_heads  =  4
stack_size  =  1
total_time_steps  =  192
num_encoder_steps  =  168
num_epochs  =  1
early_stopping_patience  =  5
multiprocessing_workers  =  5
column_definition  =  [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('power_usage', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('hour', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_

## Calculating Mean Absolute Percentage Error ( MAPE )

In [7]:
from sklearn.metrics import mean_absolute_percentage_error

print("*** Running tests ***")
tf.reset_default_graph()

with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    best_params = opt_manager.get_best_params()
    model = ModelClass(best_params, use_cudnn=use_gpu)

    model.load(opt_manager.hyperparam_folder)

    print("Computing best validation loss")
    val_loss = model.evaluate(valid)

    print("Computing test loss")
    output_map = model.predict(test, return_targets=True)
    targets = data_formatter.format_predictions(output_map["targets"])

    p10_forecast = data_formatter.format_predictions(output_map["p10"])
    p50_forecast = data_formatter.format_predictions(output_map["p50"])
    p90_forecast = data_formatter.format_predictions(output_map["p90"])

    def extract_numerical_data(data):
        """Strips out forecast time and identifier columns."""
        return data[[
            col for col in data.columns
            if col not in {"forecast_time", "identifier"}
        ]]

    p10_loss = mean_absolute_percentage_error(extract_numerical_data(targets), extract_numerical_data(p10_forecast))
    p50_loss = mean_absolute_percentage_error(extract_numerical_data(targets), extract_numerical_data(p50_forecast))
    p90_loss = mean_absolute_percentage_error(extract_numerical_data(targets), extract_numerical_data(p90_forecast))

    tf.keras.backend.set_session(default_keras_session)

print("Training completed @ {}".format(dte.datetime.now()))
print("Best validation loss = {}".format(val_loss))
print("Params:")

for k in best_params:
    print(k, " = ", best_params[k])

print("*"*100)
print("*"*100)
print("Mean Absolute Percentage Error for Test Data: P10={},  P50={}, P90={}".format(p10_loss.mean(), p50_loss.mean(), p90_loss.mean()))
print("*"*100)
print("*"*100)

loss_experiment_tracker[dataset_folder_path].update({
    "Mean Absolute Percentage Error p10 Loss": str(p10_loss.mean()),
    "Mean Absolute Percentage Error p50 Loss": str(p50_loss.mean()),
    "Mean Absolute Percentage Error p90 Loss": str(p90_loss.mean()),
})


*** Running tests ***
Resetting temp folder...
*** TemporalFusionTransformer params ***
# dropout_rate = 0.1
# hidden_layer_size = 5
# learning_rate = 0.001
# max_gradient_norm = 0.01
# minibatch_size = 64
# model_folder = electricity_dataset/saved_models/electricity/fixed
# num_heads = 4
# stack_size = 1
# total_time_steps = 192
# num_encoder_steps = 168
# num_epochs = 1
# early_stopping_patience = 5
# multiprocessing_workers = 5
# column_definition = [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('power_usage', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('hour', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('categorical_id', <DataTypes.CATEGORICAL: 1>, <InputTypes.STATIC_INPUT: 3>)]
# input_size = 5
# output_size = 1
# category_c

                                                                  'multiply_1[0][0]']             
                                                                                                  
 activation_1 (Activation)      (None, 1)            0           ['layer_normalization[0][0]']    
                                                                                                  
 layer_normalization_1 (LayerNo  (None, 1, 5)        10          ['add_1[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 1, 1)]      0           ['activation_1[0][0]']           
 sformer/ExpandDims (TensorFlow                                                                   
 OpLayer)                                                                                         
          

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 1)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_9 (Tenso                                  former/stack_1[0][0]']           
 rFlowOpLayer)                                                                                    
                                                                                                  
 add_2 (Add)                    (None, 5)            0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Sum[0][0]',               
                                                                  'multiply_3[0][0]']             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 4)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/c

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['time_distributed_27[0][0]',    
 sformer/add_11 (TensorFlowOpLa                                   'time_distributed_28[0][0]']    
 yer)                                                                                             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_16 (Tens                                  former/strided_slice_10[0][0]']  
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/s

 dropout_11 (Dropout)           (None, 24, 5)        0           ['time_distributed_29[0][0]']    
                                                                                                  
 time_distributed_33 (TimeDistr  (None, 24, 5)       30          ['activation_15[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_37 (TimeDistr  (None, 24, 5)       30          ['activation_16[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_41 (TimeDistr  (None, 24, 5)       30          ['activation_17[0][0]']          
 ibuted)                                                                                          
          

...

                                                                                                  
 layer_normalization_16 (LayerN  (None, 192, 5)      10          ['add_16[0][0]']                 
 ormalization)                                                                                    
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(3,)]              0           ['layer_normalization_16[0][0]'] 
 sformer/Shape (TensorFlowOpLay                                                                   
 er)                                                                                              
                                                                                                  
 tf_op_layer_TemporalFusionTran  [()]                0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_19 (Tens                                  former/Shape[0][0]']             
 orFlowOpL

                                                                                                  
 lambda_10 (Lambda)             (None, None, None)   0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Cumsum[0][0]']            
                                                                                                  
 add_17 (Add)                   (None, 192, 192)     0           ['lambda[0][0]',                 
                                                                  'lambda_1[0][0]']               
                                                                                                  
 add_18 (Add)                   (None, 192, 192)     0           ['lambda_3[0][0]',               
                                                                  'lambda_4[0][0]']               
                                                                                                  
 add_19 (A

 ibuted)                                                                                          
                                                                                                  
 activation_20 (Activation)     (None, 192, 5)       0           ['time_distributed_53[0][0]']    
                                                                                                  
 time_distributed_54 (TimeDistr  (None, 192, 5)      30          ['activation_20[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 dropout_24 (Dropout)           (None, 192, 5)       0           ['time_distributed_54[0][0]']    
                                                                                                  
 time_distributed_55 (TimeDistr  (None, 192, 5)      30          ['dropout_24[0][0]']             
 ibuted)  

[[-0.29020384  0.50408405  0.28548178 -0.76294833  0.44405282]
 [-0.17000875 -0.3202748   0.7630339   0.70936894  0.08873764]
 [-0.5194756  -0.56624275 -0.5694066   0.05383813  0.11093733]
 [ 0.01492704 -0.02773314 -0.5336125  -0.2008438  -0.01222838]
 [ 0.10505106 -0.4516507  -0.4922871   0.77177954 -0.20352636]]
tensor: TemporalFusionTransformer/dense_26/bias (float32) [5]
[ 1.6937578e-03 -1.8185424e-03  1.2608177e-03  1.6823163e-03
 -3.6415509e-05]
tensor: TemporalFusionTransformer/dense_26/kernel (float32) [5, 5]
[[-0.29166988  0.5055649   0.28724444 -0.7641771   0.44530174]
 [-0.17040922 -0.31966883  0.76158917  0.7083346   0.08609212]
 [-0.51934767 -0.56696093 -0.56955564  0.05513396  0.10988884]
 [ 0.01525852 -0.02872019 -0.53209907 -0.19920652 -0.01062828]
 [ 0.10380124 -0.45168796 -0.49354133  0.7717782  -0.20225443]]
tensor: TemporalFusionTransformer/dense_27/bias (float32) [5]
[-0.00181856  0.00182405 -0.00185118  0.00038008  0.00167512]
tensor: TemporalFusionTransformer/den

INFO:tensorflow:Restoring parameters from electricity_dataset/saved_models/electricity/fixed/TemporalFusionTransformer.ckpt
Done.
Computing best validation loss
Computing test loss
Training completed @ 2022-09-25 00:09:31.485920
Best validation loss = 2.3729475421444284
Params:
dropout_rate  =  0.1
hidden_layer_size  =  5
learning_rate  =  0.001
max_gradient_norm  =  0.01
minibatch_size  =  64
model_folder  =  electricity_dataset/saved_models/electricity/fixed
num_heads  =  4
stack_size  =  1
total_time_steps  =  192
num_encoder_steps  =  168
num_epochs  =  1
early_stopping_patience  =  5
multiprocessing_workers  =  5
column_definition  =  [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('power_usage', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('hour', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_

## Calculating R2 Score

In [8]:
from sklearn.metrics import r2_score

print("*** Running tests ***")
tf.reset_default_graph()

with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    best_params = opt_manager.get_best_params()
    model = ModelClass(best_params, use_cudnn=use_gpu)

    model.load(opt_manager.hyperparam_folder)

    print("Computing best validation loss")
    val_loss = model.evaluate(valid)

    print("Computing test loss")
    output_map = model.predict(test, return_targets=True)
    targets = data_formatter.format_predictions(output_map["targets"])

    p10_forecast = data_formatter.format_predictions(output_map["p10"])
    p50_forecast = data_formatter.format_predictions(output_map["p50"])
    p90_forecast = data_formatter.format_predictions(output_map["p90"])

    def extract_numerical_data(data):
        """Strips out forecast time and identifier columns."""
        return data[[
            col for col in data.columns
            if col not in {"forecast_time", "identifier"}
        ]]

    p10_loss = r2_score(extract_numerical_data(targets), extract_numerical_data(p10_forecast))
    p50_loss = r2_score(extract_numerical_data(targets), extract_numerical_data(p50_forecast))
    p90_loss = r2_score(extract_numerical_data(targets), extract_numerical_data(p90_forecast))

    tf.keras.backend.set_session(default_keras_session)

print("Training completed @ {}".format(dte.datetime.now()))
print("Best validation loss = {}".format(val_loss))
print("Params:")

for k in best_params:
    print(k, " = ", best_params[k])

print("*"*100)
print("*"*100)
print("R2 Error for Test Data: P10={},  P50={}, P90={}".format(p10_loss.mean(), p50_loss.mean(), p90_loss.mean()))
print("*"*100)
print("*"*100)

loss_experiment_tracker[dataset_folder_path].update({
    "R2 Error p10 Loss": str(p10_loss.mean()),
    "R2 Error p50 Loss": str(p50_loss.mean()),
    "R2 Error p90 Loss": str(p90_loss.mean()),
})


*** Running tests ***
Resetting temp folder...
*** TemporalFusionTransformer params ***
# dropout_rate = 0.1
# hidden_layer_size = 5
# learning_rate = 0.001
# max_gradient_norm = 0.01
# minibatch_size = 64
# model_folder = electricity_dataset/saved_models/electricity/fixed
# num_heads = 4
# stack_size = 1
# total_time_steps = 192
# num_encoder_steps = 168
# num_epochs = 1
# early_stopping_patience = 5
# multiprocessing_workers = 5
# column_definition = [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('power_usage', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('hour', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('categorical_id', <DataTypes.CATEGORICAL: 1>, <InputTypes.STATIC_INPUT: 3>)]
# input_size = 5
# output_size = 1
# category_c

                                                                  'multiply_1[0][0]']             
                                                                                                  
 activation_1 (Activation)      (None, 1)            0           ['layer_normalization[0][0]']    
                                                                                                  
 layer_normalization_1 (LayerNo  (None, 1, 5)        10          ['add_1[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 1, 1)]      0           ['activation_1[0][0]']           
 sformer/ExpandDims (TensorFlow                                                                   
 OpLayer)                                                                                         
          

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 1)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_9 (Tenso                                  former/stack_1[0][0]']           
 rFlowOpLayer)                                                                                    
                                                                                                  
 add_2 (Add)                    (None, 5)            0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Sum[0][0]',               
                                                                  'multiply_3[0][0]']             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 4)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/c

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['time_distributed_27[0][0]',    
 sformer/add_11 (TensorFlowOpLa                                   'time_distributed_28[0][0]']    
 yer)                                                                                             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_16 (Tens                                  former/strided_slice_10[0][0]']  
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/s

 dropout_11 (Dropout)           (None, 24, 5)        0           ['time_distributed_29[0][0]']    
                                                                                                  
 time_distributed_33 (TimeDistr  (None, 24, 5)       30          ['activation_15[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_37 (TimeDistr  (None, 24, 5)       30          ['activation_16[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_41 (TimeDistr  (None, 24, 5)       30          ['activation_17[0][0]']          
 ibuted)                                                                                          
          

...

                                                                                                  
 lambda_10 (Lambda)             (None, None, None)   0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Cumsum[0][0]']            
                                                                                                  
 add_17 (Add)                   (None, 192, 192)     0           ['lambda[0][0]',                 
                                                                  'lambda_1[0][0]']               
                                                                                                  
 add_18 (Add)                   (None, 192, 192)     0           ['lambda_3[0][0]',               
                                                                  'lambda_4[0][0]']               
                                                                                                  
 add_19 (A

 ibuted)                                                                                          
                                                                                                  
 activation_20 (Activation)     (None, 192, 5)       0           ['time_distributed_53[0][0]']    
                                                                                                  
 time_distributed_54 (TimeDistr  (None, 192, 5)      30          ['activation_20[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 dropout_24 (Dropout)           (None, 192, 5)       0           ['time_distributed_54[0][0]']    
                                                                                                  
 time_distributed_55 (TimeDistr  (None, 192, 5)      30          ['dropout_24[0][0]']             
 ibuted)  

[[-0.29116356  0.5051246   0.28694078 -0.7637354   0.44514504]
 [-0.16987237 -0.32045755  0.76225215  0.70940095  0.08766437]
 [-0.5201368  -0.56589764 -0.5700784   0.05353443  0.11050908]
 [ 0.01543191 -0.02798475 -0.5329644  -0.20050763 -0.01259818]
 [ 0.10338096 -0.45221442 -0.49260145  0.7723361  -0.20177114]]
tensor: TemporalFusionTransformer/dense_23/bias (float32) [5]
[-4.9927481e-04  6.2140269e-04 -7.6708628e-04 -9.3754941e-05
  2.0772844e-04]
tensor: TemporalFusionTransformer/dense_23/kernel (float32) [5, 5]
[[-0.29143846  0.5056497   0.28635058 -0.76488245  0.44584802]
 [-0.16960697 -0.32098666  0.7620938   0.70891255  0.08761343]
 [-0.5193534  -0.56623685 -0.5698193   0.05365813  0.11019721]
 [ 0.01504875 -0.02752768 -0.5323942  -0.20028251 -0.01265366]
 [ 0.10384276 -0.4526502  -0.49265462  0.77263    -0.20178433]]
tensor: TemporalFusionTransformer/dense_24/bias (float32) [5]
[-3.8455048e-05 -1.7964248e-04 -3.3144755e-04 -3.1674388e-04
  2.8680192e-04]
tensor: TemporalFusio

[[-0.32399     0.56617904  0.3184147 ]
 [-0.85247475  0.49960873 -0.19139238]
 [-0.3611128   0.8543981   0.7943001 ]
 [ 0.0967494  -0.5828188  -0.6312461 ]
 [-0.638856    0.05993678  0.12433241]]
tensor: TemporalFusionTransformer/time_distributed_31/bias (float32) [3]
[-0.00131823 -0.00060808  0.00120043]
tensor: TemporalFusionTransformer/time_distributed_31/kernel (float32) [5, 3]
[[-0.324024    0.56720465  0.3184438 ]
 [-0.8527962   0.49991715 -0.19139382]
 [-0.35847214  0.85421216  0.79146475]
 [ 0.09698287 -0.58209956 -0.6311433 ]
 [-0.6365676   0.0608546   0.12097427]]
tensor: TemporalFusionTransformer/time_distributed_32/bias (float32) [5]
[ 0.00191894 -0.001993    0.0019453   0.00198779  0.00072371]
tensor: TemporalFusionTransformer/time_distributed_32/kernel (float32) [5, 5]
[[-0.28960475  0.5074725   0.2879064  -0.7661747   0.4434685 ]
 [-0.1713683  -0.3231617   0.7610158   0.71166474  0.08903688]
 [-0.5214634  -0.56819373 -0.57123715  0.05589708  0.11175485]
 [ 0.01700031 -0.

[[-0.29312345  0.50681365  0.28765    -0.76480395  0.44717792]
 [-0.1713952  -0.31966773  0.7628732   0.7093791   0.08910273]
 [-0.5211713  -0.56435883 -0.5679194   0.05509486  0.110283  ]
 [ 0.01703205 -0.02901639 -0.53282106 -0.20126992 -0.01409081]
 [ 0.10191391 -0.4509121  -0.49307388  0.77299404 -0.20030569]]
tensor: TemporalFusionTransformer/time_distributed_8/bias (float32) [4]
[ 0.00055606 -0.00199542  0.00198199 -0.00052934]
tensor: TemporalFusionTransformer/time_distributed_8/kernel (float32) [5, 4]
[[-0.3078651   0.53097934  0.30400404 -0.80532   ]
 [ 0.46982056 -0.17664167 -0.33788434  0.80220956]
 [ 0.74848616  0.09002417 -0.5495461  -0.595026  ]
 [-0.6007717   0.05501096  0.11753315  0.01759244]
 [-0.02821125 -0.56023365 -0.21373586 -0.01302417]]
tensor: TemporalFusionTransformer/time_distributed_9/bias (float32) [4]
[ 0.00036357 -0.00189868  0.00190561 -0.00193576]
tensor: TemporalFusionTransformer/time_distributed_9/kernel (float32) [5, 4]
[[-0.30544168  0.53484344  0.3

## Calculating Mean Squared Error (MSE)

In [9]:
from sklearn.metrics import mean_squared_error

print("*** Running tests ***")
tf.reset_default_graph()

with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    best_params = opt_manager.get_best_params()
    model = ModelClass(best_params, use_cudnn=use_gpu)

    model.load(opt_manager.hyperparam_folder)

    print("Computing best validation loss")
    val_loss = model.evaluate(valid)

    print("Computing test loss")
    output_map = model.predict(test, return_targets=True)
    targets = data_formatter.format_predictions(output_map["targets"])

    p10_forecast = data_formatter.format_predictions(output_map["p10"])
    p50_forecast = data_formatter.format_predictions(output_map["p50"])
    p90_forecast = data_formatter.format_predictions(output_map["p90"])

    def extract_numerical_data(data):
        """Strips out forecast time and identifier columns."""
        return data[[
            col for col in data.columns
            if col not in {"forecast_time", "identifier"}
        ]]

    p10_loss = mean_squared_error(extract_numerical_data(targets), extract_numerical_data(p10_forecast))
    p50_loss = mean_squared_error(extract_numerical_data(targets), extract_numerical_data(p50_forecast))
    p90_loss = mean_squared_error(extract_numerical_data(targets), extract_numerical_data(p90_forecast))

    tf.keras.backend.set_session(default_keras_session)

print("Training completed @ {}".format(dte.datetime.now()))
print("Best validation loss = {}".format(val_loss))
print("Params:")

for k in best_params:
    print(k, " = ", best_params[k])

print("*"*100)
print("*"*100)
print("Mean Squared Error Loss for Test Data: P10={},  P50={}, P90={}".format(p10_loss.mean(), p50_loss.mean(), p90_loss.mean()))
print("*"*100)
print("*"*100)

loss_experiment_tracker[dataset_folder_path].update({
    "Mean Squared Error p10 Loss": str(p10_loss.mean()),
    "Mean Squared Error p50 Loss": str(p50_loss.mean()),
    "Mean Squared Error p90 Loss": str(p90_loss.mean()),
})

*** Running tests ***
Resetting temp folder...
*** TemporalFusionTransformer params ***
# dropout_rate = 0.1
# hidden_layer_size = 5
# learning_rate = 0.001
# max_gradient_norm = 0.01
# minibatch_size = 64
# model_folder = electricity_dataset/saved_models/electricity/fixed
# num_heads = 4
# stack_size = 1
# total_time_steps = 192
# num_encoder_steps = 168
# num_epochs = 1
# early_stopping_patience = 5
# multiprocessing_workers = 5
# column_definition = [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('power_usage', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('hour', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('categorical_id', <DataTypes.CATEGORICAL: 1>, <InputTypes.STATIC_INPUT: 3>)]
# input_size = 5
# output_size = 1
# category_c

                                                                  'multiply_1[0][0]']             
                                                                                                  
 activation_1 (Activation)      (None, 1)            0           ['layer_normalization[0][0]']    
                                                                                                  
 layer_normalization_1 (LayerNo  (None, 1, 5)        10          ['add_1[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 1, 1)]      0           ['activation_1[0][0]']           
 sformer/ExpandDims (TensorFlow                                                                   
 OpLayer)                                                                                         
          

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 1)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_9 (Tenso                                  former/stack_1[0][0]']           
 rFlowOpLayer)                                                                                    
                                                                                                  
 add_2 (Add)                    (None, 5)            0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Sum[0][0]',               
                                                                  'multiply_3[0][0]']             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 4)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/c

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['time_distributed_27[0][0]',    
 sformer/add_11 (TensorFlowOpLa                                   'time_distributed_28[0][0]']    
 yer)                                                                                             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_16 (Tens                                  former/strided_slice_10[0][0]']  
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/s

 dropout_11 (Dropout)           (None, 24, 5)        0           ['time_distributed_29[0][0]']    
                                                                                                  
 time_distributed_33 (TimeDistr  (None, 24, 5)       30          ['activation_15[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_37 (TimeDistr  (None, 24, 5)       30          ['activation_16[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_41 (TimeDistr  (None, 24, 5)       30          ['activation_17[0][0]']          
 ibuted)                                                                                          
          

...

                                                                 former/Sum_2[0][0]',             
                                                                  'cu_dnnlstm[0][1]',             
                                                                  'cu_dnnlstm[0][2]']             
                                                                                                  
 dense_18 (Dense)               (None, 5)            30          ['activation_4[0][0]']           
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 192, 5)]    0           ['cu_dnnlstm[0][0]',             
 sformer/concat_2 (TensorFlowOp                                   'cu_dnnlstm_1[0][0]']           
 Layer)                                                                                           
                                                                                                  
 dropout_3

                                                                                                  
 layer_normalization_16 (LayerN  (None, 192, 5)      10          ['add_16[0][0]']                 
 ormalization)                                                                                    
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(3,)]              0           ['layer_normalization_16[0][0]'] 
 sformer/Shape (TensorFlowOpLay                                                                   
 er)                                                                                              
                                                                                                  
 tf_op_layer_TemporalFusionTran  [()]                0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_19 (Tens                                  former/Shape[0][0]']             
 orFlowOpL

                                                                                                  
 lambda_10 (Lambda)             (None, None, None)   0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Cumsum[0][0]']            
                                                                                                  
 add_17 (Add)                   (None, 192, 192)     0           ['lambda[0][0]',                 
                                                                  'lambda_1[0][0]']               
                                                                                                  
 add_18 (Add)                   (None, 192, 192)     0           ['lambda_3[0][0]',               
                                                                  'lambda_4[0][0]']               
                                                                                                  
 add_19 (A

 ibuted)                                                                                          
                                                                                                  
 activation_20 (Activation)     (None, 192, 5)       0           ['time_distributed_53[0][0]']    
                                                                                                  
 time_distributed_54 (TimeDistr  (None, 192, 5)      30          ['activation_20[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 dropout_24 (Dropout)           (None, 192, 5)       0           ['time_distributed_54[0][0]']    
                                                                                                  
 time_distributed_55 (TimeDistr  (None, 192, 5)      30          ['dropout_24[0][0]']             
 ibuted)  

INFO:tensorflow:Restoring parameters from electricity_dataset/saved_models/electricity/fixed/TemporalFusionTransformer.ckpt
Done.
Computing best validation loss
Computing test loss
Training completed @ 2022-09-25 00:26:02.578754
Best validation loss = 2.3729475421444284
Params:
dropout_rate  =  0.1
hidden_layer_size  =  5
learning_rate  =  0.001
max_gradient_norm  =  0.01
minibatch_size  =  64
model_folder  =  electricity_dataset/saved_models/electricity/fixed
num_heads  =  4
stack_size  =  1
total_time_steps  =  192
num_encoder_steps  =  168
num_epochs  =  1
early_stopping_patience  =  5
multiprocessing_workers  =  5
column_definition  =  [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('power_usage', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('hour', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_

## Calculating Root Mean Squared Error (RMSE)

In [10]:
from sklearn.metrics import mean_squared_error

print("*** Running tests ***")
tf.reset_default_graph()

with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    best_params = opt_manager.get_best_params()
    model = ModelClass(best_params, use_cudnn=use_gpu)

    model.load(opt_manager.hyperparam_folder)

    print("Computing best validation loss")
    val_loss = model.evaluate(valid)

    print("Computing test loss")
    output_map = model.predict(test, return_targets=True)
    targets = data_formatter.format_predictions(output_map["targets"])

    p10_forecast = data_formatter.format_predictions(output_map["p10"])
    p50_forecast = data_formatter.format_predictions(output_map["p50"])
    p90_forecast = data_formatter.format_predictions(output_map["p90"])

    def extract_numerical_data(data):
        """Strips out forecast time and identifier columns."""
        return data[[
            col for col in data.columns
            if col not in {"forecast_time", "identifier"}
        ]]
    
    import math
    p10_loss = math.sqrt(mean_squared_error(extract_numerical_data(targets), extract_numerical_data(p10_forecast)))
    p50_loss = math.sqrt(mean_squared_error(extract_numerical_data(targets), extract_numerical_data(p50_forecast)))
    p90_loss = math.sqrt(mean_squared_error(extract_numerical_data(targets), extract_numerical_data(p90_forecast)))

    tf.keras.backend.set_session(default_keras_session)

print("Training completed @ {}".format(dte.datetime.now()))
print("Best validation loss = {}".format(val_loss))
print("Params:")

for k in best_params:
    print(k, " = ", best_params[k])

print("*"*100)
print("*"*100)
print("Root Mean Squared Error Loss for Test Data: P10={},  P50={}, P90={}".format(p10_loss, p50_loss, p90_loss))
print("*"*100)
print("*"*100)

loss_experiment_tracker[dataset_folder_path].update({
    "Root Mean Squared Error p10 Loss": str(p10_loss),
    "Root Mean Squared Error p50 Loss": str(p50_loss),
    "Root Mean Squared Error p90 Loss": str(p90_loss),
})

*** Running tests ***
Resetting temp folder...
*** TemporalFusionTransformer params ***
# dropout_rate = 0.1
# hidden_layer_size = 5
# learning_rate = 0.001
# max_gradient_norm = 0.01
# minibatch_size = 64
# model_folder = electricity_dataset/saved_models/electricity/fixed
# num_heads = 4
# stack_size = 1
# total_time_steps = 192
# num_encoder_steps = 168
# num_epochs = 1
# early_stopping_patience = 5
# multiprocessing_workers = 5
# column_definition = [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('power_usage', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('hour', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('categorical_id', <DataTypes.CATEGORICAL: 1>, <InputTypes.STATIC_INPUT: 3>)]
# input_size = 5
# output_size = 1
# category_c

                                                                  'multiply_1[0][0]']             
                                                                                                  
 activation_1 (Activation)      (None, 1)            0           ['layer_normalization[0][0]']    
                                                                                                  
 layer_normalization_1 (LayerNo  (None, 1, 5)        10          ['add_1[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 1, 1)]      0           ['activation_1[0][0]']           
 sformer/ExpandDims (TensorFlow                                                                   
 OpLayer)                                                                                         
          

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 1)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_9 (Tenso                                  former/stack_1[0][0]']           
 rFlowOpLayer)                                                                                    
                                                                                                  
 add_2 (Add)                    (None, 5)            0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Sum[0][0]',               
                                                                  'multiply_3[0][0]']             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 4)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/c

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['time_distributed_27[0][0]',    
 sformer/add_11 (TensorFlowOpLa                                   'time_distributed_28[0][0]']    
 yer)                                                                                             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_16 (Tens                                  former/strided_slice_10[0][0]']  
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/s

 dropout_11 (Dropout)           (None, 24, 5)        0           ['time_distributed_29[0][0]']    
                                                                                                  
 time_distributed_33 (TimeDistr  (None, 24, 5)       30          ['activation_15[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_37 (TimeDistr  (None, 24, 5)       30          ['activation_16[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_41 (TimeDistr  (None, 24, 5)       30          ['activation_17[0][0]']          
 ibuted)                                                                                          
          

...

                                                                 former/Sum_2[0][0]',             
                                                                  'cu_dnnlstm[0][1]',             
                                                                  'cu_dnnlstm[0][2]']             
                                                                                                  
 dense_18 (Dense)               (None, 5)            30          ['activation_4[0][0]']           
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 192, 5)]    0           ['cu_dnnlstm[0][0]',             
 sformer/concat_2 (TensorFlowOp                                   'cu_dnnlstm_1[0][0]']           
 Layer)                                                                                           
                                                                                                  
 dropout_3

                                                                                                  
 layer_normalization_16 (LayerN  (None, 192, 5)      10          ['add_16[0][0]']                 
 ormalization)                                                                                    
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(3,)]              0           ['layer_normalization_16[0][0]'] 
 sformer/Shape (TensorFlowOpLay                                                                   
 er)                                                                                              
                                                                                                  
 tf_op_layer_TemporalFusionTran  [()]                0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_19 (Tens                                  former/Shape[0][0]']             
 orFlowOpL

                                                                                                  
 lambda_10 (Lambda)             (None, None, None)   0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Cumsum[0][0]']            
                                                                                                  
 add_17 (Add)                   (None, 192, 192)     0           ['lambda[0][0]',                 
                                                                  'lambda_1[0][0]']               
                                                                                                  
 add_18 (Add)                   (None, 192, 192)     0           ['lambda_3[0][0]',               
                                                                  'lambda_4[0][0]']               
                                                                                                  
 add_19 (A

 ibuted)                                                                                          
                                                                                                  
 activation_20 (Activation)     (None, 192, 5)       0           ['time_distributed_53[0][0]']    
                                                                                                  
 time_distributed_54 (TimeDistr  (None, 192, 5)      30          ['activation_20[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 dropout_24 (Dropout)           (None, 192, 5)       0           ['time_distributed_54[0][0]']    
                                                                                                  
 time_distributed_55 (TimeDistr  (None, 192, 5)      30          ['dropout_24[0][0]']             
 ibuted)  

INFO:tensorflow:Restoring parameters from electricity_dataset/saved_models/electricity/fixed/TemporalFusionTransformer.ckpt
Done.
Computing best validation loss
Computing test loss
Training completed @ 2022-09-25 00:34:57.339709
Best validation loss = 2.3729475421444284
Params:
dropout_rate  =  0.1
hidden_layer_size  =  5
learning_rate  =  0.001
max_gradient_norm  =  0.01
minibatch_size  =  64
model_folder  =  electricity_dataset/saved_models/electricity/fixed
num_heads  =  4
stack_size  =  1
total_time_steps  =  192
num_encoder_steps  =  168
num_epochs  =  1
early_stopping_patience  =  5
multiprocessing_workers  =  5
column_definition  =  [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('power_usage', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('hour', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_

## Calculating Relative Squared Error (RSE)

In [11]:
def relative_squared_error(true, pred):
    true_mean = np.mean(true)
    squared_error_num = np.sum(np.square(true - pred))
    squared_error_den = np.sum(np.square(true - true_mean))
    rse_loss = squared_error_num / squared_error_den
    return rse_loss

print("*** Running tests ***")
tf.reset_default_graph()

with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    best_params = opt_manager.get_best_params()
    model = ModelClass(best_params, use_cudnn=use_gpu)

    model.load(opt_manager.hyperparam_folder)

    print("Computing best validation loss")
    val_loss = model.evaluate(valid)

    print("Computing test loss")
    output_map = model.predict(test, return_targets=True)
    targets = data_formatter.format_predictions(output_map["targets"])

    p10_forecast = data_formatter.format_predictions(output_map["p10"])
    p50_forecast = data_formatter.format_predictions(output_map["p50"])
    p90_forecast = data_formatter.format_predictions(output_map["p90"])

    def extract_numerical_data(data):
        """Strips out forecast time and identifier columns."""
        return data[[
            col for col in data.columns
            if col not in {"forecast_time", "identifier"}
        ]]

    p10_loss = relative_squared_error(extract_numerical_data(targets), extract_numerical_data(p10_forecast))
    p50_loss = relative_squared_error(extract_numerical_data(targets), extract_numerical_data(p50_forecast))
    p90_loss = relative_squared_error(extract_numerical_data(targets), extract_numerical_data(p90_forecast))

    tf.keras.backend.set_session(default_keras_session)

print("Training completed @ {}".format(dte.datetime.now()))
print("Best validation loss = {}".format(val_loss))
print("Params:")

for k in best_params:
    print(k, " = ", best_params[k])

print("*"*100)
print("*"*100)
print("Relative Squared Error Loss for Test Data: P10={},  P50={}, P90={}".format(p10_loss.mean(), p50_loss.mean(), p90_loss.mean()))
print("*"*100)
print("*"*100)

loss_experiment_tracker[dataset_folder_path].update({
    "Relative Squared Error p10 Loss": str(p10_loss.mean()),
    "Relative Squared Error p50 Loss": str(p50_loss.mean()),
    "Relative Squared Error p90 Loss": str(p90_loss.mean()),
})

*** Running tests ***
Resetting temp folder...
*** TemporalFusionTransformer params ***
# dropout_rate = 0.1
# hidden_layer_size = 5
# learning_rate = 0.001
# max_gradient_norm = 0.01
# minibatch_size = 64
# model_folder = electricity_dataset/saved_models/electricity/fixed
# num_heads = 4
# stack_size = 1
# total_time_steps = 192
# num_encoder_steps = 168
# num_epochs = 1
# early_stopping_patience = 5
# multiprocessing_workers = 5
# column_definition = [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('power_usage', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('hour', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('categorical_id', <DataTypes.CATEGORICAL: 1>, <InputTypes.STATIC_INPUT: 3>)]
# input_size = 5
# output_size = 1
# category_c

                                                                  'multiply_1[0][0]']             
                                                                                                  
 activation_1 (Activation)      (None, 1)            0           ['layer_normalization[0][0]']    
                                                                                                  
 layer_normalization_1 (LayerNo  (None, 1, 5)        10          ['add_1[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 1, 1)]      0           ['activation_1[0][0]']           
 sformer/ExpandDims (TensorFlow                                                                   
 OpLayer)                                                                                         
          

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 1)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_9 (Tenso                                  former/stack_1[0][0]']           
 rFlowOpLayer)                                                                                    
                                                                                                  
 add_2 (Add)                    (None, 5)            0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Sum[0][0]',               
                                                                  'multiply_3[0][0]']             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 4)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/c

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['time_distributed_27[0][0]',    
 sformer/add_11 (TensorFlowOpLa                                   'time_distributed_28[0][0]']    
 yer)                                                                                             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_16 (Tens                                  former/strided_slice_10[0][0]']  
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/s

 dropout_11 (Dropout)           (None, 24, 5)        0           ['time_distributed_29[0][0]']    
                                                                                                  
 time_distributed_33 (TimeDistr  (None, 24, 5)       30          ['activation_15[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_37 (TimeDistr  (None, 24, 5)       30          ['activation_16[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_41 (TimeDistr  (None, 24, 5)       30          ['activation_17[0][0]']          
 ibuted)                                                                                          
          

...

                                                                 former/Sum_2[0][0]',             
                                                                  'cu_dnnlstm[0][1]',             
                                                                  'cu_dnnlstm[0][2]']             
                                                                                                  
 dense_18 (Dense)               (None, 5)            30          ['activation_4[0][0]']           
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 192, 5)]    0           ['cu_dnnlstm[0][0]',             
 sformer/concat_2 (TensorFlowOp                                   'cu_dnnlstm_1[0][0]']           
 Layer)                                                                                           
                                                                                                  
 dropout_3

                                                                                                  
 layer_normalization_16 (LayerN  (None, 192, 5)      10          ['add_16[0][0]']                 
 ormalization)                                                                                    
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(3,)]              0           ['layer_normalization_16[0][0]'] 
 sformer/Shape (TensorFlowOpLay                                                                   
 er)                                                                                              
                                                                                                  
 tf_op_layer_TemporalFusionTran  [()]                0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_19 (Tens                                  former/Shape[0][0]']             
 orFlowOpL

                                                                                                  
 lambda_10 (Lambda)             (None, None, None)   0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Cumsum[0][0]']            
                                                                                                  
 add_17 (Add)                   (None, 192, 192)     0           ['lambda[0][0]',                 
                                                                  'lambda_1[0][0]']               
                                                                                                  
 add_18 (Add)                   (None, 192, 192)     0           ['lambda_3[0][0]',               
                                                                  'lambda_4[0][0]']               
                                                                                                  
 add_19 (A

 ibuted)                                                                                          
                                                                                                  
 activation_20 (Activation)     (None, 192, 5)       0           ['time_distributed_53[0][0]']    
                                                                                                  
 time_distributed_54 (TimeDistr  (None, 192, 5)      30          ['activation_20[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 dropout_24 (Dropout)           (None, 192, 5)       0           ['time_distributed_54[0][0]']    
                                                                                                  
 time_distributed_55 (TimeDistr  (None, 192, 5)      30          ['dropout_24[0][0]']             
 ibuted)  

INFO:tensorflow:Restoring parameters from electricity_dataset/saved_models/electricity/fixed/TemporalFusionTransformer.ckpt
Done.
Computing best validation loss
Computing test loss
Training completed @ 2022-09-25 00:42:51.814539
Best validation loss = 2.3729475421444284
Params:
dropout_rate  =  0.1
hidden_layer_size  =  5
learning_rate  =  0.001
max_gradient_norm  =  0.01
minibatch_size  =  64
model_folder  =  electricity_dataset/saved_models/electricity/fixed
num_heads  =  4
stack_size  =  1
total_time_steps  =  192
num_encoder_steps  =  168
num_epochs  =  1
early_stopping_patience  =  5
multiprocessing_workers  =  5
column_definition  =  [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('power_usage', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('hour', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_

## Calculating Mean Squared Logarithmic Error ( MSLE )

In [13]:
"""
from sklearn.metrics import mean_squared_log_error

print("*** Running tests ***")
tf.reset_default_graph()

with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    best_params = opt_manager.get_best_params()
    model = ModelClass(best_params, use_cudnn=use_gpu)

    model.load(opt_manager.hyperparam_folder)

    print("Computing best validation loss")
    val_loss = model.evaluate(valid)

    print("Computing test loss")
    output_map = model.predict(test, return_targets=True)
    targets = data_formatter.format_predictions(output_map["targets"])

    p10_forecast = data_formatter.format_predictions(output_map["p10"])
    p50_forecast = data_formatter.format_predictions(output_map["p50"])
    p90_forecast = data_formatter.format_predictions(output_map["p90"])

    def extract_numerical_data(data):
        '''Strips out forecast time and identifier columns.'''
        return data[[
            col for col in data.columns
            if col not in {"forecast_time", "identifier"}
        ]]

    p10_loss = mean_squared_log_error(extract_numerical_data(targets), extract_numerical_data(p10_forecast))
    p50_loss = mean_squared_log_error(extract_numerical_data(targets), extract_numerical_data(p50_forecast))
    p90_loss = mean_squared_log_error(extract_numerical_data(targets), extract_numerical_data(p90_forecast))

    tf.keras.backend.set_session(default_keras_session)

print("Training completed @ {}".format(dte.datetime.now()))
print("Best validation loss = {}".format(val_loss))
print("Params:")

for k in best_params:
    print(k, " = ", best_params[k])

print("*"*100)
print("*"*100)
print("Mean Squared Log Error Loss for Test Data: P10={},  P50={}, P90={}".format(p10_loss.mean(), p50_loss.mean(), p90_loss.mean()))
print("*"*100)
print("*"*100)

loss_experiment_tracker[dataset_folder_path].update({
    "Mean Squared Log Error p10 Loss": str(p10_loss.mean()),
    "Mean Squared Log Error p50 Loss": str(p50_loss.mean()),
    "Mean Squared Log Error p90 Loss": str(p90_loss.mean()),
})
"""

'\nfrom sklearn.metrics import mean_squared_log_error\n\nprint("*** Running tests ***")\ntf.reset_default_graph()\n\nwith tf.Graph().as_default(), tf.Session(config=tf_config) as sess:\n\n    tf.keras.backend.set_session(sess)\n    best_params = opt_manager.get_best_params()\n    model = ModelClass(best_params, use_cudnn=use_gpu)\n\n    model.load(opt_manager.hyperparam_folder)\n\n    print("Computing best validation loss")\n    val_loss = model.evaluate(valid)\n\n    print("Computing test loss")\n    output_map = model.predict(test, return_targets=True)\n    targets = data_formatter.format_predictions(output_map["targets"])\n\n    p10_forecast = data_formatter.format_predictions(output_map["p10"])\n    p50_forecast = data_formatter.format_predictions(output_map["p50"])\n    p90_forecast = data_formatter.format_predictions(output_map["p90"])\n\n    def extract_numerical_data(data):\n        \'\'\'Strips out forecast time and identifier columns.\'\'\'\n        return data[[\n           

## Calculating Root Mean Squared Logarithmic Error ( RMSLE )

In [14]:
"""
from sklearn.metrics import mean_squared_log_error
import math 

print("*** Running tests ***")
tf.reset_default_graph()

with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    best_params = opt_manager.get_best_params()
    model = ModelClass(best_params, use_cudnn=use_gpu)

    model.load(opt_manager.hyperparam_folder)

    print("Computing best validation loss")
    val_loss = model.evaluate(valid)

    print("Computing test loss")
    output_map = model.predict(test, return_targets=True)
    targets = data_formatter.format_predictions(output_map["targets"])

    p10_forecast = data_formatter.format_predictions(output_map["p10"])
    p50_forecast = data_formatter.format_predictions(output_map["p50"])
    p90_forecast = data_formatter.format_predictions(output_map["p90"])

    def extract_numerical_data(data):
        '''Strips out forecast time and identifier columns.'''
        return data[[
            col for col in data.columns
            if col not in {"forecast_time", "identifier"}
        ]]
    
    p10_loss = math.sqrt(mean_squared_log_error(extract_numerical_data(targets), extract_numerical_data(p10_forecast)))
    p50_loss = math.sqrt(mean_squared_log_error(extract_numerical_data(targets), extract_numerical_data(p50_forecast)))
    p90_loss = math.sqrt(mean_squared_log_error(extract_numerical_data(targets), extract_numerical_data(p90_forecast)))

    tf.keras.backend.set_session(default_keras_session)

print("Training completed @ {}".format(dte.datetime.now()))
print("Best validation loss = {}".format(val_loss))
print("Params:")

for k in best_params:
    print(k, " = ", best_params[k])

print("*"*100)
print("*"*100)
print("Root Mean Squared Log Error Loss for Test Data: P10={},  P50={}, P90={}".format(p10_loss.mean(), p50_loss.mean(), p90_loss.mean()))
print("*"*100)
print("*"*100)

loss_experiment_tracker[dataset_folder_path].update({
    "Root Mean Squared Log Error p10 Loss": str(p10_loss.mean()),
    "Root Mean Squared Log Error p50 Loss": str(p50_loss.mean()),
    "Root Mean Squared Log Error p90 Loss": str(p90_loss.mean()),
})
"""

'\nfrom sklearn.metrics import mean_squared_log_error\nimport math \n\nprint("*** Running tests ***")\ntf.reset_default_graph()\n\nwith tf.Graph().as_default(), tf.Session(config=tf_config) as sess:\n\n    tf.keras.backend.set_session(sess)\n    best_params = opt_manager.get_best_params()\n    model = ModelClass(best_params, use_cudnn=use_gpu)\n\n    model.load(opt_manager.hyperparam_folder)\n\n    print("Computing best validation loss")\n    val_loss = model.evaluate(valid)\n\n    print("Computing test loss")\n    output_map = model.predict(test, return_targets=True)\n    targets = data_formatter.format_predictions(output_map["targets"])\n\n    p10_forecast = data_formatter.format_predictions(output_map["p10"])\n    p50_forecast = data_formatter.format_predictions(output_map["p50"])\n    p90_forecast = data_formatter.format_predictions(output_map["p90"])\n\n    def extract_numerical_data(data):\n        \'\'\'Strips out forecast time and identifier columns.\'\'\'\n        return data[

## Calculating Huber Loss, delta = 0.5

In [15]:
def huber_loss(true, pred, delta):
    huber_mse = 0.5 * np.square(true - pred)
    huber_mae = delta * (np.abs(true - pred) - 0.5 * (np.square(delta)))
    return np.where(np.abs(true - pred) <= delta, huber_mse, huber_mae)


print("*** Running tests ***")
tf.reset_default_graph()

with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    best_params = opt_manager.get_best_params()
    model = ModelClass(best_params, use_cudnn=use_gpu)

    model.load(opt_manager.hyperparam_folder)

    print("Computing best validation loss")
    val_loss = model.evaluate(valid)

    print("Computing test loss")
    output_map = model.predict(test, return_targets=True)
    targets = data_formatter.format_predictions(output_map["targets"])

    p10_forecast = data_formatter.format_predictions(output_map["p10"])
    p50_forecast = data_formatter.format_predictions(output_map["p50"])
    p90_forecast = data_formatter.format_predictions(output_map["p90"])

    def extract_numerical_data(data):
        """Strips out forecast time and identifier columns."""
        return data[[
            col for col in data.columns
            if col not in {"forecast_time", "identifier"}
        ]]
    
    p10_loss = huber_loss(extract_numerical_data(targets), extract_numerical_data(p10_forecast),0.5)
    p50_loss = huber_loss(extract_numerical_data(targets), extract_numerical_data(p50_forecast),0.5)
    p90_loss = huber_loss(extract_numerical_data(targets), extract_numerical_data(p90_forecast),0.5)

    tf.keras.backend.set_session(default_keras_session)

print("Training completed @ {}".format(dte.datetime.now()))
print("Best validation loss = {}".format(val_loss))
print("Params:")

for k in best_params:
    print(k, " = ", best_params[k])

print("*"*100)
print("*"*100)
print("Huber Loss with Delta 0.5 for Test Data: P10={},  P50={}, P90={}".format(p10_loss.mean(), p50_loss.mean(), p90_loss.mean()))
print("*"*100)
print("*"*100)

loss_experiment_tracker[dataset_folder_path].update({
    "Huber Loss Delta with-0.5 p10 Loss": str(p10_loss.mean()),
    "Huber Loss Delta with-0.5 p50 Loss": str(p50_loss.mean()),
    "Huber Loss Delta with-0.5 p90 Loss": str(p90_loss.mean()),
})


*** Running tests ***
Resetting temp folder...
*** TemporalFusionTransformer params ***
# dropout_rate = 0.1
# hidden_layer_size = 5
# learning_rate = 0.001
# max_gradient_norm = 0.01
# minibatch_size = 64
# model_folder = electricity_dataset/saved_models/electricity/fixed
# num_heads = 4
# stack_size = 1
# total_time_steps = 192
# num_encoder_steps = 168
# num_epochs = 1
# early_stopping_patience = 5
# multiprocessing_workers = 5
# column_definition = [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('power_usage', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('hour', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('categorical_id', <DataTypes.CATEGORICAL: 1>, <InputTypes.STATIC_INPUT: 3>)]
# input_size = 5
# output_size = 1
# category_c

                                                                  'multiply_1[0][0]']             
                                                                                                  
 activation_1 (Activation)      (None, 1)            0           ['layer_normalization[0][0]']    
                                                                                                  
 layer_normalization_1 (LayerNo  (None, 1, 5)        10          ['add_1[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 1, 1)]      0           ['activation_1[0][0]']           
 sformer/ExpandDims (TensorFlow                                                                   
 OpLayer)                                                                                         
          

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 1)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_9 (Tenso                                  former/stack_1[0][0]']           
 rFlowOpLayer)                                                                                    
                                                                                                  
 add_2 (Add)                    (None, 5)            0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Sum[0][0]',               
                                                                  'multiply_3[0][0]']             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 4)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/c

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['time_distributed_27[0][0]',    
 sformer/add_11 (TensorFlowOpLa                                   'time_distributed_28[0][0]']    
 yer)                                                                                             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_16 (Tens                                  former/strided_slice_10[0][0]']  
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/s

 dropout_11 (Dropout)           (None, 24, 5)        0           ['time_distributed_29[0][0]']    
                                                                                                  
 time_distributed_33 (TimeDistr  (None, 24, 5)       30          ['activation_15[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_37 (TimeDistr  (None, 24, 5)       30          ['activation_16[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_41 (TimeDistr  (None, 24, 5)       30          ['activation_17[0][0]']          
 ibuted)                                                                                          
          

...

                                                                 former/Sum_2[0][0]',             
                                                                  'cu_dnnlstm[0][1]',             
                                                                  'cu_dnnlstm[0][2]']             
                                                                                                  
 dense_18 (Dense)               (None, 5)            30          ['activation_4[0][0]']           
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 192, 5)]    0           ['cu_dnnlstm[0][0]',             
 sformer/concat_2 (TensorFlowOp                                   'cu_dnnlstm_1[0][0]']           
 Layer)                                                                                           
                                                                                                  
 dropout_3

                                                                                                  
 layer_normalization_16 (LayerN  (None, 192, 5)      10          ['add_16[0][0]']                 
 ormalization)                                                                                    
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(3,)]              0           ['layer_normalization_16[0][0]'] 
 sformer/Shape (TensorFlowOpLay                                                                   
 er)                                                                                              
                                                                                                  
 tf_op_layer_TemporalFusionTran  [()]                0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_19 (Tens                                  former/Shape[0][0]']             
 orFlowOpL

                                                                                                  
 lambda_10 (Lambda)             (None, None, None)   0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Cumsum[0][0]']            
                                                                                                  
 add_17 (Add)                   (None, 192, 192)     0           ['lambda[0][0]',                 
                                                                  'lambda_1[0][0]']               
                                                                                                  
 add_18 (Add)                   (None, 192, 192)     0           ['lambda_3[0][0]',               
                                                                  'lambda_4[0][0]']               
                                                                                                  
 add_19 (A

 ibuted)                                                                                          
                                                                                                  
 activation_20 (Activation)     (None, 192, 5)       0           ['time_distributed_53[0][0]']    
                                                                                                  
 time_distributed_54 (TimeDistr  (None, 192, 5)      30          ['activation_20[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 dropout_24 (Dropout)           (None, 192, 5)       0           ['time_distributed_54[0][0]']    
                                                                                                  
 time_distributed_55 (TimeDistr  (None, 192, 5)      30          ['dropout_24[0][0]']             
 ibuted)  

INFO:tensorflow:Restoring parameters from electricity_dataset/saved_models/electricity/fixed/TemporalFusionTransformer.ckpt
Done.
Computing best validation loss
Computing test loss
Training completed @ 2022-09-25 01:01:29.929555
Best validation loss = 2.3729475421444284
Params:
dropout_rate  =  0.1
hidden_layer_size  =  5
learning_rate  =  0.001
max_gradient_norm  =  0.01
minibatch_size  =  64
model_folder  =  electricity_dataset/saved_models/electricity/fixed
num_heads  =  4
stack_size  =  1
total_time_steps  =  192
num_encoder_steps  =  168
num_epochs  =  1
early_stopping_patience  =  5
multiprocessing_workers  =  5
column_definition  =  [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('power_usage', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('hour', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_

## Calculating Huber Loss, delta = 1

In [16]:
print("*** Running tests ***")
tf.reset_default_graph()

with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    best_params = opt_manager.get_best_params()
    model = ModelClass(best_params, use_cudnn=use_gpu)

    model.load(opt_manager.hyperparam_folder)

    print("Computing best validation loss")
    val_loss = model.evaluate(valid)

    print("Computing test loss")
    output_map = model.predict(test, return_targets=True)
    targets = data_formatter.format_predictions(output_map["targets"])

    p10_forecast = data_formatter.format_predictions(output_map["p10"])
    p50_forecast = data_formatter.format_predictions(output_map["p50"])
    p90_forecast = data_formatter.format_predictions(output_map["p90"])

    def extract_numerical_data(data):
        """Strips out forecast time and identifier columns."""
        return data[[
            col for col in data.columns
            if col not in {"forecast_time", "identifier"}
        ]]
    
    p10_loss = huber_loss(extract_numerical_data(targets), extract_numerical_data(p10_forecast),1)
    p50_loss = huber_loss(extract_numerical_data(targets), extract_numerical_data(p50_forecast),1)
    p90_loss = huber_loss(extract_numerical_data(targets), extract_numerical_data(p90_forecast),1)

    tf.keras.backend.set_session(default_keras_session)

print("Training completed @ {}".format(dte.datetime.now()))
print("Best validation loss = {}".format(val_loss))
print("Params:")

for k in best_params:
    print(k, " = ", best_params[k])

print("*"*100)
print("*"*100)
print("Huber Loss with Delta 1.0 for Test Data: P10={},  P50={}, P90={}".format(p10_loss.mean(), p50_loss.mean(), p90_loss.mean()))
print("*"*100)
print("*"*100)

loss_experiment_tracker[dataset_folder_path].update({
    "Huber Loss Delta with-1.0 p10 Loss": str(p10_loss.mean()),
    "Huber Loss Delta with-1.0 p50 Loss": str(p50_loss.mean()),
    "Huber Loss Delta with-1.0 p90 Loss": str(p90_loss.mean()),
})


*** Running tests ***
Resetting temp folder...
*** TemporalFusionTransformer params ***
# dropout_rate = 0.1
# hidden_layer_size = 5
# learning_rate = 0.001
# max_gradient_norm = 0.01
# minibatch_size = 64
# model_folder = electricity_dataset/saved_models/electricity/fixed
# num_heads = 4
# stack_size = 1
# total_time_steps = 192
# num_encoder_steps = 168
# num_epochs = 1
# early_stopping_patience = 5
# multiprocessing_workers = 5
# column_definition = [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('power_usage', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('hour', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('categorical_id', <DataTypes.CATEGORICAL: 1>, <InputTypes.STATIC_INPUT: 3>)]
# input_size = 5
# output_size = 1
# category_c

                                                                  'multiply_1[0][0]']             
                                                                                                  
 activation_1 (Activation)      (None, 1)            0           ['layer_normalization[0][0]']    
                                                                                                  
 layer_normalization_1 (LayerNo  (None, 1, 5)        10          ['add_1[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 1, 1)]      0           ['activation_1[0][0]']           
 sformer/ExpandDims (TensorFlow                                                                   
 OpLayer)                                                                                         
          

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 1)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_9 (Tenso                                  former/stack_1[0][0]']           
 rFlowOpLayer)                                                                                    
                                                                                                  
 add_2 (Add)                    (None, 5)            0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Sum[0][0]',               
                                                                  'multiply_3[0][0]']             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 4)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/c

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['time_distributed_27[0][0]',    
 sformer/add_11 (TensorFlowOpLa                                   'time_distributed_28[0][0]']    
 yer)                                                                                             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_16 (Tens                                  former/strided_slice_10[0][0]']  
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/s

 dropout_11 (Dropout)           (None, 24, 5)        0           ['time_distributed_29[0][0]']    
                                                                                                  
 time_distributed_33 (TimeDistr  (None, 24, 5)       30          ['activation_15[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_37 (TimeDistr  (None, 24, 5)       30          ['activation_16[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_41 (TimeDistr  (None, 24, 5)       30          ['activation_17[0][0]']          
 ibuted)                                                                                          
          

...

                                                                                                  
 layer_normalization_16 (LayerN  (None, 192, 5)      10          ['add_16[0][0]']                 
 ormalization)                                                                                    
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(3,)]              0           ['layer_normalization_16[0][0]'] 
 sformer/Shape (TensorFlowOpLay                                                                   
 er)                                                                                              
                                                                                                  
 tf_op_layer_TemporalFusionTran  [()]                0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_19 (Tens                                  former/Shape[0][0]']             
 orFlowOpL

                                                                                                  
 lambda_10 (Lambda)             (None, None, None)   0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Cumsum[0][0]']            
                                                                                                  
 add_17 (Add)                   (None, 192, 192)     0           ['lambda[0][0]',                 
                                                                  'lambda_1[0][0]']               
                                                                                                  
 add_18 (Add)                   (None, 192, 192)     0           ['lambda_3[0][0]',               
                                                                  'lambda_4[0][0]']               
                                                                                                  
 add_19 (A

 ibuted)                                                                                          
                                                                                                  
 activation_20 (Activation)     (None, 192, 5)       0           ['time_distributed_53[0][0]']    
                                                                                                  
 time_distributed_54 (TimeDistr  (None, 192, 5)      30          ['activation_20[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 dropout_24 (Dropout)           (None, 192, 5)       0           ['time_distributed_54[0][0]']    
                                                                                                  
 time_distributed_55 (TimeDistr  (None, 192, 5)      30          ['dropout_24[0][0]']             
 ibuted)  

[[-0.37839207]
 [ 0.65120065]
 [ 0.36816204]
 [-0.9882293 ]
 [ 0.5769392 ]]
tensor: TemporalFusionTransformer/dense_78/kernel (float32) [5, 1]
[[-0.3782262 ]
 [ 0.6546534 ]
 [ 0.37184513]
 [-0.98855036]
 [ 0.57391036]]
tensor: TemporalFusionTransformer/dense_79/kernel (float32) [5, 1]
[[-0.37834203]
 [ 0.651164  ]
 [ 0.368162  ]
 [-0.98622   ]
 [ 0.5769261 ]]
tensor: TemporalFusionTransformer/dense_8/bias (float32) [1]
[0.]
tensor: TemporalFusionTransformer/dense_8/kernel (float32) [5, 1]
[[-0.37641406]
 [ 0.65268254]
 [ 0.36989117]
 [-0.9865818 ]
 [ 0.5749903 ]]
tensor: TemporalFusionTransformer/dense_80/kernel (float32) [5, 1]
[[-0.37822634]
 [ 0.65465516]
 [ 0.37183625]
 [-0.98854977]
 [ 0.57393044]]
tensor: TemporalFusionTransformer/dense_81/kernel (float32) [5, 1]
[[-0.37837553]
 [ 0.6511455 ]
 [ 0.36816186]
 [-0.9861858 ]
 [ 0.5769333 ]]
tensor: TemporalFusionTransformer/dense_82/kernel (float32) [5, 1]
[[-0.3781831 ]
 [ 0.6546464 ]
 [ 0.37181756]
 [-0.98850423]
 [ 0.5738259 ]]
t

INFO:tensorflow:Restoring parameters from electricity_dataset/saved_models/electricity/fixed/TemporalFusionTransformer.ckpt
Done.
Computing best validation loss
Computing test loss
Training completed @ 2022-09-25 01:09:27.460235
Best validation loss = 2.3729475421444284
Params:
dropout_rate  =  0.1
hidden_layer_size  =  5
learning_rate  =  0.001
max_gradient_norm  =  0.01
minibatch_size  =  64
model_folder  =  electricity_dataset/saved_models/electricity/fixed
num_heads  =  4
stack_size  =  1
total_time_steps  =  192
num_encoder_steps  =  168
num_epochs  =  1
early_stopping_patience  =  5
multiprocessing_workers  =  5
column_definition  =  [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('power_usage', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('hour', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_

## Calculating Huber Loss, delta = 5

In [17]:
print("*** Running tests ***")
tf.reset_default_graph()

with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    best_params = opt_manager.get_best_params()
    model = ModelClass(best_params, use_cudnn=use_gpu)

    model.load(opt_manager.hyperparam_folder)

    print("Computing best validation loss")
    val_loss = model.evaluate(valid)

    print("Computing test loss")
    output_map = model.predict(test, return_targets=True)
    targets = data_formatter.format_predictions(output_map["targets"])

    p10_forecast = data_formatter.format_predictions(output_map["p10"])
    p50_forecast = data_formatter.format_predictions(output_map["p50"])
    p90_forecast = data_formatter.format_predictions(output_map["p90"])

    def extract_numerical_data(data):
        """Strips out forecast time and identifier columns."""
        return data[[
            col for col in data.columns
            if col not in {"forecast_time", "identifier"}
        ]]
    
    p10_loss = huber_loss(extract_numerical_data(targets), extract_numerical_data(p10_forecast),5)
    p50_loss = huber_loss(extract_numerical_data(targets), extract_numerical_data(p50_forecast),5)
    p90_loss = huber_loss(extract_numerical_data(targets), extract_numerical_data(p90_forecast),5)

    tf.keras.backend.set_session(default_keras_session)

print("Training completed @ {}".format(dte.datetime.now()))
print("Best validation loss = {}".format(val_loss))
print("Params:")

for k in best_params:
    print(k, " = ", best_params[k])

print("*"*100)
print("*"*100)
print("Huber Loss with Delta 5.0 for Test Data: P10={},  P50={}, P90={}".format(p10_loss.mean(), p50_loss.mean(), p90_loss.mean()))
print("*"*100)
print("*"*100)

loss_experiment_tracker[dataset_folder_path].update({
    "Huber Loss Delta with-5.0 p10 Loss": str(p10_loss.mean()),
    "Huber Loss Delta with-5.0 p50 Loss": str(p50_loss.mean()),
    "Huber Loss Delta with-5.0 p90 Loss": str(p90_loss.mean()),
})


*** Running tests ***
Resetting temp folder...
*** TemporalFusionTransformer params ***
# dropout_rate = 0.1
# hidden_layer_size = 5
# learning_rate = 0.001
# max_gradient_norm = 0.01
# minibatch_size = 64
# model_folder = electricity_dataset/saved_models/electricity/fixed
# num_heads = 4
# stack_size = 1
# total_time_steps = 192
# num_encoder_steps = 168
# num_epochs = 1
# early_stopping_patience = 5
# multiprocessing_workers = 5
# column_definition = [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('power_usage', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('hour', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('categorical_id', <DataTypes.CATEGORICAL: 1>, <InputTypes.STATIC_INPUT: 3>)]
# input_size = 5
# output_size = 1
# category_c

                                                                  'multiply_1[0][0]']             
                                                                                                  
 activation_1 (Activation)      (None, 1)            0           ['layer_normalization[0][0]']    
                                                                                                  
 layer_normalization_1 (LayerNo  (None, 1, 5)        10          ['add_1[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 1, 1)]      0           ['activation_1[0][0]']           
 sformer/ExpandDims (TensorFlow                                                                   
 OpLayer)                                                                                         
          

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 1)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_9 (Tenso                                  former/stack_1[0][0]']           
 rFlowOpLayer)                                                                                    
                                                                                                  
 add_2 (Add)                    (None, 5)            0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Sum[0][0]',               
                                                                  'multiply_3[0][0]']             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 4)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/c

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['time_distributed_27[0][0]',    
 sformer/add_11 (TensorFlowOpLa                                   'time_distributed_28[0][0]']    
 yer)                                                                                             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_16 (Tens                                  former/strided_slice_10[0][0]']  
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/s

 dropout_11 (Dropout)           (None, 24, 5)        0           ['time_distributed_29[0][0]']    
                                                                                                  
 time_distributed_33 (TimeDistr  (None, 24, 5)       30          ['activation_15[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_37 (TimeDistr  (None, 24, 5)       30          ['activation_16[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_41 (TimeDistr  (None, 24, 5)       30          ['activation_17[0][0]']          
 ibuted)                                                                                          
          

...

                                                                 former/Sum_2[0][0]',             
                                                                  'cu_dnnlstm[0][1]',             
                                                                  'cu_dnnlstm[0][2]']             
                                                                                                  
 dense_18 (Dense)               (None, 5)            30          ['activation_4[0][0]']           
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 192, 5)]    0           ['cu_dnnlstm[0][0]',             
 sformer/concat_2 (TensorFlowOp                                   'cu_dnnlstm_1[0][0]']           
 Layer)                                                                                           
                                                                                                  
 dropout_3

                                                                                                  
 layer_normalization_16 (LayerN  (None, 192, 5)      10          ['add_16[0][0]']                 
 ormalization)                                                                                    
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(3,)]              0           ['layer_normalization_16[0][0]'] 
 sformer/Shape (TensorFlowOpLay                                                                   
 er)                                                                                              
                                                                                                  
 tf_op_layer_TemporalFusionTran  [()]                0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_19 (Tens                                  former/Shape[0][0]']             
 orFlowOpL

                                                                                                  
 lambda_10 (Lambda)             (None, None, None)   0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Cumsum[0][0]']            
                                                                                                  
 add_17 (Add)                   (None, 192, 192)     0           ['lambda[0][0]',                 
                                                                  'lambda_1[0][0]']               
                                                                                                  
 add_18 (Add)                   (None, 192, 192)     0           ['lambda_3[0][0]',               
                                                                  'lambda_4[0][0]']               
                                                                                                  
 add_19 (A

 ibuted)                                                                                          
                                                                                                  
 activation_20 (Activation)     (None, 192, 5)       0           ['time_distributed_53[0][0]']    
                                                                                                  
 time_distributed_54 (TimeDistr  (None, 192, 5)      30          ['activation_20[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 dropout_24 (Dropout)           (None, 192, 5)       0           ['time_distributed_54[0][0]']    
                                                                                                  
 time_distributed_55 (TimeDistr  (None, 192, 5)      30          ['dropout_24[0][0]']             
 ibuted)  

INFO:tensorflow:Restoring parameters from electricity_dataset/saved_models/electricity/fixed/TemporalFusionTransformer.ckpt
Done.
Computing best validation loss
Computing test loss
Training completed @ 2022-09-25 01:17:18.721210
Best validation loss = 2.3729475421444284
Params:
dropout_rate  =  0.1
hidden_layer_size  =  5
learning_rate  =  0.001
max_gradient_norm  =  0.01
minibatch_size  =  64
model_folder  =  electricity_dataset/saved_models/electricity/fixed
num_heads  =  4
stack_size  =  1
total_time_steps  =  192
num_encoder_steps  =  168
num_epochs  =  1
early_stopping_patience  =  5
multiprocessing_workers  =  5
column_definition  =  [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('power_usage', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('hour', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_

## Calculating Log Cosh Loss

In [18]:
def log_cosh(true, pred):
    logcosh = np.log(np.cosh(pred - true))
    logcosh_loss = np.sum(logcosh)
    return logcosh_loss


print("*** Running tests ***")
tf.reset_default_graph()

with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    best_params = opt_manager.get_best_params()
    model = ModelClass(best_params, use_cudnn=use_gpu)

    model.load(opt_manager.hyperparam_folder)

    print("Computing best validation loss")
    val_loss = model.evaluate(valid)

    print("Computing test loss")
    output_map = model.predict(test, return_targets=True)
    targets = data_formatter.format_predictions(output_map["targets"])

    p10_forecast = data_formatter.format_predictions(output_map["p10"])
    p50_forecast = data_formatter.format_predictions(output_map["p50"])
    p90_forecast = data_formatter.format_predictions(output_map["p90"])

    def extract_numerical_data(data):
        """Strips out forecast time and identifier columns."""
        return data[[
            col for col in data.columns
            if col not in {"forecast_time", "identifier"}
        ]]
    
    p10_loss = log_cosh(extract_numerical_data(targets), extract_numerical_data(p10_forecast))
    p50_loss = log_cosh(extract_numerical_data(targets), extract_numerical_data(p50_forecast))
    p90_loss = log_cosh(extract_numerical_data(targets), extract_numerical_data(p90_forecast))

    tf.keras.backend.set_session(default_keras_session)

print("Training completed @ {}".format(dte.datetime.now()))
print("Best validation loss = {}".format(val_loss))
print("Params:")

for k in best_params:
    print(k, " = ", best_params[k])

print("*"*100)
print("*"*100)
print("Log Cosh Loss for Test Data: P10={},  P50={}, P90={}".format(p10_loss.mean(), p50_loss.mean(), p90_loss.mean()))
print("*"*100)
print("*"*100)

loss_experiment_tracker[dataset_folder_path].update({
    "Log Cosh Loss p10 Loss": str(p10_loss.mean()),
    "Log Cosh Loss p50 Loss": str(p50_loss.mean()),
    "Log Cosh Loss p90 Loss": str(p90_loss.mean()),
})


*** Running tests ***
Resetting temp folder...
*** TemporalFusionTransformer params ***
# dropout_rate = 0.1
# hidden_layer_size = 5
# learning_rate = 0.001
# max_gradient_norm = 0.01
# minibatch_size = 64
# model_folder = electricity_dataset/saved_models/electricity/fixed
# num_heads = 4
# stack_size = 1
# total_time_steps = 192
# num_encoder_steps = 168
# num_epochs = 1
# early_stopping_patience = 5
# multiprocessing_workers = 5
# column_definition = [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('power_usage', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('hour', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('categorical_id', <DataTypes.CATEGORICAL: 1>, <InputTypes.STATIC_INPUT: 3>)]
# input_size = 5
# output_size = 1
# category_c

                                                                  'multiply_1[0][0]']             
                                                                                                  
 activation_1 (Activation)      (None, 1)            0           ['layer_normalization[0][0]']    
                                                                                                  
 layer_normalization_1 (LayerNo  (None, 1, 5)        10          ['add_1[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 1, 1)]      0           ['activation_1[0][0]']           
 sformer/ExpandDims (TensorFlow                                                                   
 OpLayer)                                                                                         
          

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 1)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_9 (Tenso                                  former/stack_1[0][0]']           
 rFlowOpLayer)                                                                                    
                                                                                                  
 add_2 (Add)                    (None, 5)            0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Sum[0][0]',               
                                                                  'multiply_3[0][0]']             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 4)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/c

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['time_distributed_27[0][0]',    
 sformer/add_11 (TensorFlowOpLa                                   'time_distributed_28[0][0]']    
 yer)                                                                                             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_16 (Tens                                  former/strided_slice_10[0][0]']  
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/s

 dropout_11 (Dropout)           (None, 24, 5)        0           ['time_distributed_29[0][0]']    
                                                                                                  
 time_distributed_33 (TimeDistr  (None, 24, 5)       30          ['activation_15[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_37 (TimeDistr  (None, 24, 5)       30          ['activation_16[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_41 (TimeDistr  (None, 24, 5)       30          ['activation_17[0][0]']          
 ibuted)                                                                                          
          

...

                                                                 former/Sum_2[0][0]',             
                                                                  'cu_dnnlstm[0][1]',             
                                                                  'cu_dnnlstm[0][2]']             
                                                                                                  
 dense_18 (Dense)               (None, 5)            30          ['activation_4[0][0]']           
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 192, 5)]    0           ['cu_dnnlstm[0][0]',             
 sformer/concat_2 (TensorFlowOp                                   'cu_dnnlstm_1[0][0]']           
 Layer)                                                                                           
                                                                                                  
 dropout_3

                                                                                                  
 layer_normalization_16 (LayerN  (None, 192, 5)      10          ['add_16[0][0]']                 
 ormalization)                                                                                    
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(3,)]              0           ['layer_normalization_16[0][0]'] 
 sformer/Shape (TensorFlowOpLay                                                                   
 er)                                                                                              
                                                                                                  
 tf_op_layer_TemporalFusionTran  [()]                0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_19 (Tens                                  former/Shape[0][0]']             
 orFlowOpL

                                                                                                  
 lambda_10 (Lambda)             (None, None, None)   0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Cumsum[0][0]']            
                                                                                                  
 add_17 (Add)                   (None, 192, 192)     0           ['lambda[0][0]',                 
                                                                  'lambda_1[0][0]']               
                                                                                                  
 add_18 (Add)                   (None, 192, 192)     0           ['lambda_3[0][0]',               
                                                                  'lambda_4[0][0]']               
                                                                                                  
 add_19 (A

 ibuted)                                                                                          
                                                                                                  
 activation_20 (Activation)     (None, 192, 5)       0           ['time_distributed_53[0][0]']    
                                                                                                  
 time_distributed_54 (TimeDistr  (None, 192, 5)      30          ['activation_20[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 dropout_24 (Dropout)           (None, 192, 5)       0           ['time_distributed_54[0][0]']    
                                                                                                  
 time_distributed_55 (TimeDistr  (None, 192, 5)      30          ['dropout_24[0][0]']             
 ibuted)  

INFO:tensorflow:Restoring parameters from electricity_dataset/saved_models/electricity/fixed/TemporalFusionTransformer.ckpt
Done.
Computing best validation loss
Computing test loss
Training completed @ 2022-09-25 01:25:35.810644
Best validation loss = 2.3729475421444284
Params:
dropout_rate  =  0.1
hidden_layer_size  =  5
learning_rate  =  0.001
max_gradient_norm  =  0.01
minibatch_size  =  64
model_folder  =  electricity_dataset/saved_models/electricity/fixed
num_heads  =  4
stack_size  =  1
total_time_steps  =  192
num_encoder_steps  =  168
num_epochs  =  1
early_stopping_patience  =  5
multiprocessing_workers  =  5
column_definition  =  [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('power_usage', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('hour', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_

## Implementing Quantile Loss , gamma=0.25

In [19]:
def quantile_loss(true, pred, gamma):
    val1 = gamma * np.abs(true - pred)
    val2 = (1-gamma) * np.abs(true - pred)
    q_loss = np.where(true >= pred, val1, val2)
    return q_loss


print("*** Running tests ***")
tf.reset_default_graph()

with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    best_params = opt_manager.get_best_params()
    model = ModelClass(best_params, use_cudnn=use_gpu)

    model.load(opt_manager.hyperparam_folder)

    print("Computing best validation loss")
    val_loss = model.evaluate(valid)

    print("Computing test loss")
    output_map = model.predict(test, return_targets=True)
    targets = data_formatter.format_predictions(output_map["targets"])

    p10_forecast = data_formatter.format_predictions(output_map["p10"])
    p50_forecast = data_formatter.format_predictions(output_map["p50"])
    p90_forecast = data_formatter.format_predictions(output_map["p90"])

    def extract_numerical_data(data):
        """Strips out forecast time and identifier columns."""
        return data[[
            col for col in data.columns
            if col not in {"forecast_time", "identifier"}
        ]]
    
    p10_loss = quantile_loss(extract_numerical_data(targets), extract_numerical_data(p10_forecast),0.25)
    p50_loss = quantile_loss(extract_numerical_data(targets), extract_numerical_data(p50_forecast),0.25)
    p90_loss = quantile_loss(extract_numerical_data(targets), extract_numerical_data(p90_forecast),0.25)

    tf.keras.backend.set_session(default_keras_session)

print("Training completed @ {}".format(dte.datetime.now()))
print("Best validation loss = {}".format(val_loss))
print("Params:")

for k in best_params:
    print(k, " = ", best_params[k])

print("*"*100)
print("*"*100)
print("Quantile Loss with Gamma 0.25 for Test Data: P10={},  P50={}, P90={}".format(p10_loss.mean(), p50_loss.mean(), p90_loss.mean()))
print("*"*100)
print("*"*100)

loss_experiment_tracker[dataset_folder_path].update({
    "Quantile Loss Gamma with-0.25 p10 Loss": str(p10_loss.mean()),
    "Quantile Loss Gamma with-0.25 p50 Loss": str(p50_loss.mean()),
    "Quantile Loss Gamma with-0.25 p90 Loss": str(p90_loss.mean()),
})


*** Running tests ***
Resetting temp folder...
*** TemporalFusionTransformer params ***
# dropout_rate = 0.1
# hidden_layer_size = 5
# learning_rate = 0.001
# max_gradient_norm = 0.01
# minibatch_size = 64
# model_folder = electricity_dataset/saved_models/electricity/fixed
# num_heads = 4
# stack_size = 1
# total_time_steps = 192
# num_encoder_steps = 168
# num_epochs = 1
# early_stopping_patience = 5
# multiprocessing_workers = 5
# column_definition = [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('power_usage', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('hour', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('categorical_id', <DataTypes.CATEGORICAL: 1>, <InputTypes.STATIC_INPUT: 3>)]
# input_size = 5
# output_size = 1
# category_c

                                                                  'multiply_1[0][0]']             
                                                                                                  
 activation_1 (Activation)      (None, 1)            0           ['layer_normalization[0][0]']    
                                                                                                  
 layer_normalization_1 (LayerNo  (None, 1, 5)        10          ['add_1[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 1, 1)]      0           ['activation_1[0][0]']           
 sformer/ExpandDims (TensorFlow                                                                   
 OpLayer)                                                                                         
          

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 1)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_9 (Tenso                                  former/stack_1[0][0]']           
 rFlowOpLayer)                                                                                    
                                                                                                  
 add_2 (Add)                    (None, 5)            0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Sum[0][0]',               
                                                                  'multiply_3[0][0]']             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 4)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/c

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['time_distributed_27[0][0]',    
 sformer/add_11 (TensorFlowOpLa                                   'time_distributed_28[0][0]']    
 yer)                                                                                             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_16 (Tens                                  former/strided_slice_10[0][0]']  
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/s

 dropout_11 (Dropout)           (None, 24, 5)        0           ['time_distributed_29[0][0]']    
                                                                                                  
 time_distributed_33 (TimeDistr  (None, 24, 5)       30          ['activation_15[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_37 (TimeDistr  (None, 24, 5)       30          ['activation_16[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_41 (TimeDistr  (None, 24, 5)       30          ['activation_17[0][0]']          
 ibuted)                                                                                          
          

...

                                                                 former/Sum_2[0][0]',             
                                                                  'cu_dnnlstm[0][1]',             
                                                                  'cu_dnnlstm[0][2]']             
                                                                                                  
 dense_18 (Dense)               (None, 5)            30          ['activation_4[0][0]']           
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 192, 5)]    0           ['cu_dnnlstm[0][0]',             
 sformer/concat_2 (TensorFlowOp                                   'cu_dnnlstm_1[0][0]']           
 Layer)                                                                                           
                                                                                                  
 dropout_3

                                                                                                  
 layer_normalization_16 (LayerN  (None, 192, 5)      10          ['add_16[0][0]']                 
 ormalization)                                                                                    
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(3,)]              0           ['layer_normalization_16[0][0]'] 
 sformer/Shape (TensorFlowOpLay                                                                   
 er)                                                                                              
                                                                                                  
 tf_op_layer_TemporalFusionTran  [()]                0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_19 (Tens                                  former/Shape[0][0]']             
 orFlowOpL

                                                                                                  
 lambda_10 (Lambda)             (None, None, None)   0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Cumsum[0][0]']            
                                                                                                  
 add_17 (Add)                   (None, 192, 192)     0           ['lambda[0][0]',                 
                                                                  'lambda_1[0][0]']               
                                                                                                  
 add_18 (Add)                   (None, 192, 192)     0           ['lambda_3[0][0]',               
                                                                  'lambda_4[0][0]']               
                                                                                                  
 add_19 (A

 ibuted)                                                                                          
                                                                                                  
 activation_20 (Activation)     (None, 192, 5)       0           ['time_distributed_53[0][0]']    
                                                                                                  
 time_distributed_54 (TimeDistr  (None, 192, 5)      30          ['activation_20[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 dropout_24 (Dropout)           (None, 192, 5)       0           ['time_distributed_54[0][0]']    
                                                                                                  
 time_distributed_55 (TimeDistr  (None, 192, 5)      30          ['dropout_24[0][0]']             
 ibuted)  

INFO:tensorflow:Restoring parameters from electricity_dataset/saved_models/electricity/fixed/TemporalFusionTransformer.ckpt
Done.
Computing best validation loss
Computing test loss
Training completed @ 2022-09-25 01:33:31.704152
Best validation loss = 2.3729475421444284
Params:
dropout_rate  =  0.1
hidden_layer_size  =  5
learning_rate  =  0.001
max_gradient_norm  =  0.01
minibatch_size  =  64
model_folder  =  electricity_dataset/saved_models/electricity/fixed
num_heads  =  4
stack_size  =  1
total_time_steps  =  192
num_encoder_steps  =  168
num_epochs  =  1
early_stopping_patience  =  5
multiprocessing_workers  =  5
column_definition  =  [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('power_usage', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('hour', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_

## Implementing Quantile Loss , gamma=0.5

In [20]:
print("*** Running tests ***")
tf.reset_default_graph()

with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    best_params = opt_manager.get_best_params()
    model = ModelClass(best_params, use_cudnn=use_gpu)

    model.load(opt_manager.hyperparam_folder)

    print("Computing best validation loss")
    val_loss = model.evaluate(valid)

    print("Computing test loss")
    output_map = model.predict(test, return_targets=True)
    targets = data_formatter.format_predictions(output_map["targets"])

    p10_forecast = data_formatter.format_predictions(output_map["p10"])
    p50_forecast = data_formatter.format_predictions(output_map["p50"])
    p90_forecast = data_formatter.format_predictions(output_map["p90"])

    def extract_numerical_data(data):
        """Strips out forecast time and identifier columns."""
        return data[[
            col for col in data.columns
            if col not in {"forecast_time", "identifier"}
        ]]
    
    p10_loss = quantile_loss(extract_numerical_data(targets), extract_numerical_data(p10_forecast),0.50)
    p50_loss = quantile_loss(extract_numerical_data(targets), extract_numerical_data(p50_forecast),0.50)
    p90_loss = quantile_loss(extract_numerical_data(targets), extract_numerical_data(p90_forecast),0.50)

    tf.keras.backend.set_session(default_keras_session)

print("Training completed @ {}".format(dte.datetime.now()))
print("Best validation loss = {}".format(val_loss))
print("Params:")

for k in best_params:
    print(k, " = ", best_params[k])

print("*"*100)
print("*"*100)
print("Quantile Loss with Gamma 0.5 for Test Data: P10={},  P50={}, P90={}".format(p10_loss.mean(), p50_loss.mean(), p90_loss.mean()))
print("*"*100)
print("*"*100)

loss_experiment_tracker[dataset_folder_path].update({
    "Quantile Loss Gamma with-0.5 p10 Loss": str(p10_loss.mean()),
    "Quantile Loss Gamma with-0.5 p50 Loss": str(p50_loss.mean()),
    "Quantile Loss Gamma with-0.5 p90 Loss": str(p90_loss.mean()),
})

*** Running tests ***
Resetting temp folder...
*** TemporalFusionTransformer params ***
# dropout_rate = 0.1
# hidden_layer_size = 5
# learning_rate = 0.001
# max_gradient_norm = 0.01
# minibatch_size = 64
# model_folder = electricity_dataset/saved_models/electricity/fixed
# num_heads = 4
# stack_size = 1
# total_time_steps = 192
# num_encoder_steps = 168
# num_epochs = 1
# early_stopping_patience = 5
# multiprocessing_workers = 5
# column_definition = [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('power_usage', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('hour', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('categorical_id', <DataTypes.CATEGORICAL: 1>, <InputTypes.STATIC_INPUT: 3>)]
# input_size = 5
# output_size = 1
# category_c

                                                                  'multiply_1[0][0]']             
                                                                                                  
 activation_1 (Activation)      (None, 1)            0           ['layer_normalization[0][0]']    
                                                                                                  
 layer_normalization_1 (LayerNo  (None, 1, 5)        10          ['add_1[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 1, 1)]      0           ['activation_1[0][0]']           
 sformer/ExpandDims (TensorFlow                                                                   
 OpLayer)                                                                                         
          

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 1)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_9 (Tenso                                  former/stack_1[0][0]']           
 rFlowOpLayer)                                                                                    
                                                                                                  
 add_2 (Add)                    (None, 5)            0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Sum[0][0]',               
                                                                  'multiply_3[0][0]']             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 4)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/c

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['time_distributed_27[0][0]',    
 sformer/add_11 (TensorFlowOpLa                                   'time_distributed_28[0][0]']    
 yer)                                                                                             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_16 (Tens                                  former/strided_slice_10[0][0]']  
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/s

 dropout_11 (Dropout)           (None, 24, 5)        0           ['time_distributed_29[0][0]']    
                                                                                                  
 time_distributed_33 (TimeDistr  (None, 24, 5)       30          ['activation_15[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_37 (TimeDistr  (None, 24, 5)       30          ['activation_16[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_41 (TimeDistr  (None, 24, 5)       30          ['activation_17[0][0]']          
 ibuted)                                                                                          
          

...

                                                                 former/Sum_2[0][0]',             
                                                                  'cu_dnnlstm[0][1]',             
                                                                  'cu_dnnlstm[0][2]']             
                                                                                                  
 dense_18 (Dense)               (None, 5)            30          ['activation_4[0][0]']           
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 192, 5)]    0           ['cu_dnnlstm[0][0]',             
 sformer/concat_2 (TensorFlowOp                                   'cu_dnnlstm_1[0][0]']           
 Layer)                                                                                           
                                                                                                  
 dropout_3

                                                                                                  
 layer_normalization_16 (LayerN  (None, 192, 5)      10          ['add_16[0][0]']                 
 ormalization)                                                                                    
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(3,)]              0           ['layer_normalization_16[0][0]'] 
 sformer/Shape (TensorFlowOpLay                                                                   
 er)                                                                                              
                                                                                                  
 tf_op_layer_TemporalFusionTran  [()]                0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_19 (Tens                                  former/Shape[0][0]']             
 orFlowOpL

                                                                                                  
 lambda_10 (Lambda)             (None, None, None)   0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Cumsum[0][0]']            
                                                                                                  
 add_17 (Add)                   (None, 192, 192)     0           ['lambda[0][0]',                 
                                                                  'lambda_1[0][0]']               
                                                                                                  
 add_18 (Add)                   (None, 192, 192)     0           ['lambda_3[0][0]',               
                                                                  'lambda_4[0][0]']               
                                                                                                  
 add_19 (A

 ibuted)                                                                                          
                                                                                                  
 activation_20 (Activation)     (None, 192, 5)       0           ['time_distributed_53[0][0]']    
                                                                                                  
 time_distributed_54 (TimeDistr  (None, 192, 5)      30          ['activation_20[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 dropout_24 (Dropout)           (None, 192, 5)       0           ['time_distributed_54[0][0]']    
                                                                                                  
 time_distributed_55 (TimeDistr  (None, 192, 5)      30          ['dropout_24[0][0]']             
 ibuted)  

INFO:tensorflow:Restoring parameters from electricity_dataset/saved_models/electricity/fixed/TemporalFusionTransformer.ckpt
Done.
Computing best validation loss
Computing test loss
Training completed @ 2022-09-25 01:41:16.001714
Best validation loss = 2.3729475421444284
Params:
dropout_rate  =  0.1
hidden_layer_size  =  5
learning_rate  =  0.001
max_gradient_norm  =  0.01
minibatch_size  =  64
model_folder  =  electricity_dataset/saved_models/electricity/fixed
num_heads  =  4
stack_size  =  1
total_time_steps  =  192
num_encoder_steps  =  168
num_epochs  =  1
early_stopping_patience  =  5
multiprocessing_workers  =  5
column_definition  =  [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('power_usage', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('hour', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_

## Implementing Quantile Loss , gamma=0.75

In [21]:
print("*** Running tests ***")
tf.reset_default_graph()

with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    best_params = opt_manager.get_best_params()
    model = ModelClass(best_params, use_cudnn=use_gpu)

    model.load(opt_manager.hyperparam_folder)

    print("Computing best validation loss")
    val_loss = model.evaluate(valid)

    print("Computing test loss")
    output_map = model.predict(test, return_targets=True)
    targets = data_formatter.format_predictions(output_map["targets"])

    p10_forecast = data_formatter.format_predictions(output_map["p10"])
    p50_forecast = data_formatter.format_predictions(output_map["p50"])
    p90_forecast = data_formatter.format_predictions(output_map["p90"])

    def extract_numerical_data(data):
        """Strips out forecast time and identifier columns."""
        return data[[
            col for col in data.columns
            if col not in {"forecast_time", "identifier"}
        ]]
    
    p10_loss = quantile_loss(extract_numerical_data(targets), extract_numerical_data(p10_forecast),0.75)
    p50_loss = quantile_loss(extract_numerical_data(targets), extract_numerical_data(p50_forecast),0.75)
    p90_loss = quantile_loss(extract_numerical_data(targets), extract_numerical_data(p90_forecast),0.75)

    tf.keras.backend.set_session(default_keras_session)

print("Training completed @ {}".format(dte.datetime.now()))
print("Best validation loss = {}".format(val_loss))
print("Params:")

for k in best_params:
    print(k, " = ", best_params[k])

print("*"*100)
print("*"*100)
print("Quantile Loss with Gamma 0.75 for Test Data: P10={},  P50={}, P90={}".format(p10_loss.mean(), p50_loss.mean(), p90_loss.mean()))
print("*"*100)
print("*"*100)

loss_experiment_tracker[dataset_folder_path].update({
    "Quantile Loss Gamma with-0.75 p10 Loss": str(p10_loss.mean()),
    "Quantile Loss Gamma with-0.75 p50 Loss": str(p50_loss.mean()),
    "Quantile Loss Gamma with-0.75 p90 Loss": str(p90_loss.mean()),
})

*** Running tests ***
Resetting temp folder...
*** TemporalFusionTransformer params ***
# dropout_rate = 0.1
# hidden_layer_size = 5
# learning_rate = 0.001
# max_gradient_norm = 0.01
# minibatch_size = 64
# model_folder = electricity_dataset/saved_models/electricity/fixed
# num_heads = 4
# stack_size = 1
# total_time_steps = 192
# num_encoder_steps = 168
# num_epochs = 1
# early_stopping_patience = 5
# multiprocessing_workers = 5
# column_definition = [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('power_usage', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('hour', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('categorical_id', <DataTypes.CATEGORICAL: 1>, <InputTypes.STATIC_INPUT: 3>)]
# input_size = 5
# output_size = 1
# category_c

                                                                  'multiply_1[0][0]']             
                                                                                                  
 activation_1 (Activation)      (None, 1)            0           ['layer_normalization[0][0]']    
                                                                                                  
 layer_normalization_1 (LayerNo  (None, 1, 5)        10          ['add_1[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 1, 1)]      0           ['activation_1[0][0]']           
 sformer/ExpandDims (TensorFlow                                                                   
 OpLayer)                                                                                         
          

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 1)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_9 (Tenso                                  former/stack_1[0][0]']           
 rFlowOpLayer)                                                                                    
                                                                                                  
 add_2 (Add)                    (None, 5)            0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Sum[0][0]',               
                                                                  'multiply_3[0][0]']             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 4)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/c

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['time_distributed_27[0][0]',    
 sformer/add_11 (TensorFlowOpLa                                   'time_distributed_28[0][0]']    
 yer)                                                                                             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_16 (Tens                                  former/strided_slice_10[0][0]']  
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/s

 dropout_11 (Dropout)           (None, 24, 5)        0           ['time_distributed_29[0][0]']    
                                                                                                  
 time_distributed_33 (TimeDistr  (None, 24, 5)       30          ['activation_15[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_37 (TimeDistr  (None, 24, 5)       30          ['activation_16[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_41 (TimeDistr  (None, 24, 5)       30          ['activation_17[0][0]']          
 ibuted)                                                                                          
          

...

                                                                                                  
 layer_normalization_16 (LayerN  (None, 192, 5)      10          ['add_16[0][0]']                 
 ormalization)                                                                                    
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(3,)]              0           ['layer_normalization_16[0][0]'] 
 sformer/Shape (TensorFlowOpLay                                                                   
 er)                                                                                              
                                                                                                  
 tf_op_layer_TemporalFusionTran  [()]                0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_19 (Tens                                  former/Shape[0][0]']             
 orFlowOpL

                                                                                                  
 lambda_10 (Lambda)             (None, None, None)   0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Cumsum[0][0]']            
                                                                                                  
 add_17 (Add)                   (None, 192, 192)     0           ['lambda[0][0]',                 
                                                                  'lambda_1[0][0]']               
                                                                                                  
 add_18 (Add)                   (None, 192, 192)     0           ['lambda_3[0][0]',               
                                                                  'lambda_4[0][0]']               
                                                                                                  
 add_19 (A

 ibuted)                                                                                          
                                                                                                  
 activation_20 (Activation)     (None, 192, 5)       0           ['time_distributed_53[0][0]']    
                                                                                                  
 time_distributed_54 (TimeDistr  (None, 192, 5)      30          ['activation_20[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 dropout_24 (Dropout)           (None, 192, 5)       0           ['time_distributed_54[0][0]']    
                                                                                                  
 time_distributed_55 (TimeDistr  (None, 192, 5)      30          ['dropout_24[0][0]']             
 ibuted)  

[[-0.37822634]
 [ 0.65465516]
 [ 0.37183625]
 [-0.98854977]
 [ 0.57393044]]
tensor: TemporalFusionTransformer/dense_81/kernel (float32) [5, 1]
[[-0.37837553]
 [ 0.6511455 ]
 [ 0.36816186]
 [-0.9861858 ]
 [ 0.5769333 ]]
tensor: TemporalFusionTransformer/dense_82/kernel (float32) [5, 1]
[[-0.3781831 ]
 [ 0.6546464 ]
 [ 0.37181756]
 [-0.98850423]
 [ 0.5738259 ]]
tensor: TemporalFusionTransformer/dense_83/kernel (float32) [5, 1]
[[-0.37835577]
 [ 0.6512702 ]
 [ 0.36812198]
 [-0.98637635]
 [ 0.57697195]]
tensor: TemporalFusionTransformer/dense_84/kernel (float32) [5, 1]
[[-0.37823778]
 [ 0.6546492 ]
 [ 0.3718455 ]
 [-0.9885487 ]
 [ 0.573957  ]]
tensor: TemporalFusionTransformer/dense_85/kernel (float32) [1, 5]
[[-0.37455115  0.65468043  0.37189034 -0.984921    0.5769027 ]]
tensor: TemporalFusionTransformer/dense_9/bias (float32) [5]
[ 0.00114154  0.00112871  0.00178212 -0.00086755 -0.00090422]
tensor: TemporalFusionTransformer/dense_9/kernel (float32) [5, 5]
[[-0.29074883  0.506295    0.286

INFO:tensorflow:Restoring parameters from electricity_dataset/saved_models/electricity/fixed/TemporalFusionTransformer.ckpt
Done.
Computing best validation loss
Computing test loss
Training completed @ 2022-09-25 01:48:55.706611
Best validation loss = 2.3729475421444284
Params:
dropout_rate  =  0.1
hidden_layer_size  =  5
learning_rate  =  0.001
max_gradient_norm  =  0.01
minibatch_size  =  64
model_folder  =  electricity_dataset/saved_models/electricity/fixed
num_heads  =  4
stack_size  =  1
total_time_steps  =  192
num_encoder_steps  =  168
num_epochs  =  1
early_stopping_patience  =  5
multiprocessing_workers  =  5
column_definition  =  [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('power_usage', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('hour', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_

### Saving Experiments on a json file

In [22]:
with open("electricity_dataset_experiments.json", "w") as outfile:
    json.dump(loss_experiment_tracker, outfile)